In [ ]:
# ═════════════════════════════════════════════════════════════════════
# ETL FIDC V3 - EXTRAÇÃO COMPLETA DE DADOS FINANCEIROS (90+ CAMPOS)
# ═════════════════════════════════════════════════════════════════════
# Autor: Rafael Augusto
# Data: Dezembro 2025
# Objetivo: Extrair dados completos de FIDCs via API B3 para análise em BI
# Input: lista_cnpjs_fidc.csv (441 CNPJs únicos)
# Output: base_fidc_completa.csv (90+ campos por CNPJ)
# ═════════════════════════════════════════════════════════════════════

# ═════════════════════════════════════════════════════════════════════
# CÉLULA 1: IMPORTS E CONFIGURAÇÕES
# ═════════════════════════════════════════════════════════════════════

import pandas as pd
import requests
import base64
import xml.etree.ElementTree as ET
import time
import os
from typing import Dict, List, Optional

# Configurações da API B3
URL_API_BUSCA = "https://fnet.bmfbovespa.com.br/fnet/publico/pesquisarGerenciadorDocumentosDados"
URL_API_DOWNLOAD = "https://fnet.bmfbovespa.com.br/fnet/publico/downloadDocumento"
HEADERS = {"User-Agent": "Mozilla/5.0"}

# Configurações de controle
DELAY_ENTRE_REQUISICOES = 2  # segundos (recomendado: 2-3s para evitar bloqueio)
TIMEOUT_BUSCA = 10  # segundos
TIMEOUT_DOWNLOAD = 20  # segundos

print("✅ Bibliotecas importadas com sucesso!")
print(f"📊 Configurações:")
print(f"   • Delay entre requisições: {DELAY_ENTRE_REQUISICOES}s")
print(f"   • Timeout busca: {TIMEOUT_BUSCA}s")
print(f"   • Timeout download: {TIMEOUT_DOWNLOAD}s")

# ═════════════════════════════════════════════════════════════════════
# CÉLULA 2: FUNÇÕES AUXILIARES
# ═════════════════════════════════════════════════════════════════════

def limpar_tag(tag: str) -> str:
    """
    Remove namespace XML das tags.

    Exemplo:
        {urn:fidc}VL_DISPONIB → VL_DISPONIB
    """
    return tag.split('}')[-1] if '}' in tag else tag


def converter_valor(texto: str) -> float:
    """
    Converte string no formato brasileiro para float.

    Exemplos:
        "1.234.567,89" → 1234567.89
        "123,45"       → 123.45
        ""             → 0.0
        None           → 0.0
    """
    if not texto or not str(texto).strip():
        return 0.0

    try:
        # Remove pontos (separador de milhar) e troca vírgula por ponto
        texto_limpo = str(texto).replace('.', '').replace(',', '.')
        return float(texto_limpo)
    except (ValueError, AttributeError):
        return 0.0


def buscar_valor_xml(root: ET.Element, caminho: str) -> any:
    """
    Busca um valor no XML e tenta converter para float.
    Se não conseguir, retorna como string.

    Args:
        root: Elemento raiz do XML
        caminho: Caminho XPath simplificado (ex: "CRED_EXISTE/VL_SOM_DICRED_AQUIS")

    Returns:
        float ou string ou 0.0/''
    """
    elemento = root.find(f'.//{caminho}')

    if elemento is not None and elemento.text:
        # Tentar converter para float (valores monetários)
        try:
            return converter_valor(elemento.text)
        except:
            # Se falhar, retornar como string (datas, textos, etc)
            return elemento.text.strip()

    # Retornar 0.0 para campos numéricos, '' para texto
    return 0.0 if '/' in caminho else ''


print("✅ Funções auxiliares definidas:")
print("   • limpar_tag()")
print("   • converter_valor()")
print("   • buscar_valor_xml()")

# ═════════════════════════════════════════════════════════════════════
# CÉLULA 3: FUNÇÃO DE EXTRAÇÃO COMPLETA DO XML (90+ CAMPOS)
# ═════════════════════════════════════════════════════════════════════

def extrair_dados_xml_completo(xml_content: bytes) -> Dict:
    """
    Extrai TODOS os campos relevantes do XML de Informe Mensal FIDC.

    Args:
        xml_content: Conteúdo XML em bytes (já decodificado do Base64)

    Returns:
        Dicionário com 90+ campos estruturados para análise
    """
    root = ET.fromstring(xml_content)

    # ─────────────────────────────────────────────────────────────────────────
    # EXTRAÇÃO DE DADOS POR SEÇÃO
    # ─────────────────────────────────────────────────────────────────────────

    dados = {
        # ═══ SEÇÃO 1: IDENTIFICAÇÃO DO FUNDO ═══
        'CNPJ_FUNDO': buscar_valor_xml(root, 'NR_CNPJ_FUNDO'),
        'CNPJ_ADMINISTRADOR': buscar_valor_xml(root, 'NR_CNPJ_ADM'),
        'DATA_COMPETENCIA': buscar_valor_xml(root, 'DT_COMPT'),
        'TIPO_CONDOMINIO': buscar_valor_xml(root, 'TP_CONDOMINIO'),
        'FUNDO_EXCLUSIVO': buscar_valor_xml(root, 'FDO_EXCL'),
        'CLASSE_UNICA': buscar_valor_xml(root, 'CLASS_UNICA'),
        'COTISTA_VINCULADO': buscar_valor_xml(root, 'COTST_VINCUL'),

        # ═══ SEÇÃO 2: ATIVOS GERAIS ═══
        'ATIVO_TOTAL': buscar_valor_xml(root, 'VL_SOM_APLIC_ATIVO'),
        'DISPONIBILIDADES': buscar_valor_xml(root, 'VL_DISPONIB'),
        'CARTEIRA_TOTAL': buscar_valor_xml(root, 'VL_CARTEIRA'),
        'OUTROS_ATIVOS_TOTAL': buscar_valor_xml(root, 'VL_SOM_OUTROS_ATIVOS'),
        'OUTROS_ATIVOS_CURTO_PRAZO': buscar_valor_xml(root, 'OUTROS_ATIVOS/VL_OUTRO_VL_RECEB_CURPRZ'),
        'OUTROS_ATIVOS_LONGO_PRAZO': buscar_valor_xml(root, 'OUTROS_ATIVOS/VL_OUTRO_VL_RECEB_LPRAZO'),

        # ═══ SEÇÃO 3: CRÉDITOS EXISTENTES (Principal fonte de inadimplência) ═══
        'CREDITOS_ADQUIRIDOS': buscar_valor_xml(root, 'CRED_EXISTE/VL_SOM_DICRED_AQUIS'),
        'CRED_VENCIDOS_ADIMPLENTES': buscar_valor_xml(root, 'CRED_EXISTE/VL_CRED_EXISTE_VENC_ADIMPL'),
        'CRED_VENCIDOS_INADIMPLENTES': buscar_valor_xml(root, 'CRED_EXISTE/VL_CRED_EXISTE_VENC_INAD'),
        'CRED_TOTAL_VENC_INADIMPL': buscar_valor_xml(root, 'CRED_EXISTE/VL_CRED_TOTAL_VENC_INAD'),
        'CRED_INADIMPLENCIA': buscar_valor_xml(root, 'CRED_EXISTE/VL_CRED_EXISTE_INAD'),
        'CRED_PERFORMADOS': buscar_valor_xml(root, 'CRED_EXISTE/VL_CRED_REFER_DICRED_PERFO'),
        'CRED_VENCIDOS_PENDENTES': buscar_valor_xml(root, 'CRED_EXISTE/VL_CRED_VENC_PEND'),
        'CRED_EMP_RECUPERACAO': buscar_valor_xml(root, 'CRED_EXISTE/VL_CRED_ORIGEM_EMP_PROC_RECUP'),
        'CRED_RECEITA_PUBLICA': buscar_valor_xml(root, 'CRED_EXISTE/VL_DECOR_RECEIT_PUBLIC'),
        'CRED_ACAO_JUDICIAL': buscar_valor_xml(root, 'CRED_EXISTE/VL_CRED_ACAO_JUDIC'),
        'CRED_CONSTITUICAO_JURIDICA': buscar_valor_xml(root, 'CRED_EXISTE/VL_CRED_CONST_JUR_FATRISC'),
        'CRED_PROVISAO_REDUCAO': buscar_valor_xml(root, 'CRED_EXISTE/VL_PROVIS_REDUC_RECUP'),

        # ═══ SEÇÃO 4: DIREITOS CREDITÓRIOS (DICRED) ═══
        'DICRED_TOTAL': buscar_valor_xml(root, 'DICRED/VL_DICRED'),
        'DICRED_CEDENTE': buscar_valor_xml(root, 'DICRED/VL_DICRED_CEDENT'),
        'DICRED_VENC_INADIMPL': buscar_valor_xml(root, 'DICRED/VL_DICRED_EXISTE_VENC_INAD'),
        'DICRED_TOTAL_VENC_INAD': buscar_valor_xml(root, 'DICRED/VL_DICRED_TOTAL_VENC_INAD'),
        'DICRED_INADIMPLENCIA': buscar_valor_xml(root, 'DICRED/VL_DICRED_EXISTE_INAD'),
        'DICRED_PERFORMADOS': buscar_valor_xml(root, 'DICRED/VL_DICRED_REFER_DICRED_PERFO'),
        'DICRED_VENC_PENDENTES': buscar_valor_xml(root, 'DICRED/VL_DICRED_VENC_PEND'),
        'DICRED_EMP_RECUPERACAO': buscar_valor_xml(root, 'DICRED/VL_DICRED_ORIGEM_EMP_PROC_RECUP'),
        'DICRED_RECEITA_PUBLICA': buscar_valor_xml(root, 'DICRED/VL_DICRED_RECEIT_PUBLIC'),
        'DICRED_ACAO_JUDICIAL': buscar_valor_xml(root, 'DICRED/VL_DICRED_ACAO_JUDIC'),
        'DICRED_PROVISAO_REDUCAO': buscar_valor_xml(root, 'DICRED/VL_DICRED_PROVIS_REDUC_RECUP'),

        # ═══ SEÇÃO 5: VALORES MOBILIÁRIOS ═══
        'VALORES_MOBILIARIOS_TOTAL': buscar_valor_xml(root, 'VALORES_MOB/VL_SOM_VALORES_MOB'),
        'DEBENTURES': buscar_valor_xml(root, 'VALORES_MOB/VL_DEBT'),
        'CRI': buscar_valor_xml(root, 'VALORES_MOB/VL_CRI'),
        'NOTAS_PROMISSORIAS_COMERCIAIS': buscar_valor_xml(root, 'VALORES_MOB/VL_NP_COMERC'),
        'LETRAS_FINANCEIRAS': buscar_valor_xml(root, 'VALORES_MOB/VL_LETRA_FINANC'),
        'COTAS_FIF': buscar_valor_xml(root, 'VALORES_MOB/VL_CLS_COTA_FIF'),
        'OUTROS_DIREITOS_CREDITORIOS': buscar_valor_xml(root, 'VALORES_MOB/VL_OUTRO_DICRED'),

        # ═══ SEÇÃO 6: OUTROS ATIVOS FINANCEIROS ═══
        'TITULOS_PUBLICOS_FEDERAIS': buscar_valor_xml(root, 'VL_TITPUB_FED'),
        'CDB': buscar_valor_xml(root, 'VL_CDB'),
        'APLICACOES_COMPROMISSADAS': buscar_valor_xml(root, 'VL_APLIC_OPER_COMPSS'),
        'ATIVOS_FINANCEIROS_RF': buscar_valor_xml(root, 'VL_ATIV_FINANC_RF'),
        'COTAS_FIDC': buscar_valor_xml(root, 'VL_COTA_FIDC'),

        # ═══ SEÇÃO 7: MERCADO DE DERIVATIVOS ═══
        'DERIVATIVOS_TOTAL': buscar_valor_xml(root, 'MERC_DERIVATIVO/VL_SOM_MERC_DERIVATIVO'),
        'TERMO_COMPRADOR': buscar_valor_xml(root, 'MERC_DERIVATIVO/VL_MERC_TERMO_POS_COMPRD'),
        'OPCOES_TITULAR': buscar_valor_xml(root, 'MERC_DERIVATIVO/VL_MERC_OP_POS_TITUL'),
        'FUTUROS_AJUSTE_POSITIVO': buscar_valor_xml(root, 'MERC_DERIVATIVO/VL_MERC_FUT_AJUST_POSIT'),
        'SWAP_A_RECEBER': buscar_valor_xml(root, 'MERC_DERIVATIVO/VL_DIFER_SWAP_RECEB'),
        'COBERTURA_PRESTADA': buscar_valor_xml(root, 'MERC_DERIVATIVO/VL_COBERT_PREST'),
        'DEPOSITOS_MARGEM': buscar_valor_xml(root, 'MERC_DERIVATIVO/VL_DEPOS_MARGEM'),

        # ═══ SEÇÃO 8: SEGMENTAÇÃO DA CARTEIRA ═══
        'CARTEIRA_SEGMENTADA_TOTAL': buscar_valor_xml(root, 'CART_SEGMT/VL_SOM_CART_SEGMT'),
        'SEGMT_INDUSTRIAL': buscar_valor_xml(root, 'CART_SEGMT/VL_IND'),
        'SEGMT_MERCADO_IMOBILIARIO': buscar_valor_xml(root, 'CART_SEGMT/VL_MERC_IMOBIL'),
        'SEGMT_AGRONEGOCIO': buscar_valor_xml(root, 'CART_SEGMT/VL_AGRONEG'),
        'SEGMT_CARTAO_CREDITO': buscar_valor_xml(root, 'CART_SEGMT/VL_CART_CRED'),
        'SEGMT_ACAO_JUDICIAL': buscar_valor_xml(root, 'CART_SEGMT/VL_ACAO_JUDIC'),
        'SEGMT_PROPRIEDADE_INTELECTUAL': buscar_valor_xml(root, 'CART_SEGMT/VL_PROPRD_MARCA_PATENT'),

        # ─── Subsegmento: COMERCIAL ───
        'SEGMT_COMERCIAL_TOTAL': buscar_valor_xml(root, 'SEGMT_COMERC/VL_SOM_SEGMT_COMERC'),
        'SEGMT_COMERCIO': buscar_valor_xml(root, 'SEGMT_COMERC/VL_COMERC'),
        'SEGMT_COMERCIO_VAREJO': buscar_valor_xml(root, 'SEGMT_COMERC/VL_COMERC_VARJ'),
        'SEGMT_ARREND_MERCANTIL': buscar_valor_xml(root, 'SEGMT_COMERC/VL_ARREND_MERCNT'),

        # ─── Subsegmento: SERVIÇOS ───
        'SEGMT_SERVICOS_TOTAL': buscar_valor_xml(root, 'SEGMT_SERV/VL_SOM_SEGMT_SERV'),
        'SEGMT_SERVICOS_GERAIS': buscar_valor_xml(root, 'SEGMT_SERV/VL_SERV'),
        'SEGMT_SERVICOS_PUBLICOS': buscar_valor_xml(root, 'SEGMT_SERV/VL_SERV_PUBLIC'),
        'SEGMT_SERVICOS_EDUCACAO': buscar_valor_xml(root, 'SEGMT_SERV/VL_SERV_EDUC'),
        'SEGMT_SERVICOS_ENTRETENIMENTO': buscar_valor_xml(root, 'SEGMT_SERV/VL_SERV_ENTRETEN'),

        # ─── Subsegmento: FINANCEIRO ───
        'SEGMT_FINANCEIRO_TOTAL': buscar_valor_xml(root, 'SEGMT_FINANC/VL_SOM_SEGMT_FINANC'),
        'SEGMT_FINANC_CREDITO_PESSOA': buscar_valor_xml(root, 'SEGMT_FINANC/VL_FINANC_CRED_PESSOA'),
        'SEGMT_FINANC_CONSIGNADO': buscar_valor_xml(root, 'SEGMT_FINANC/VL_FINANC_CRED_PESSOA_CONSIG'),
        'SEGMT_FINANC_CORPORATIVO': buscar_valor_xml(root, 'SEGMT_FINANC/VL_FINANC_CRED_CORPOR'),
        'SEGMT_FINANC_MIDDLE_MARKET': buscar_valor_xml(root, 'SEGMT_FINANC/VL_FINANC_MMARKET'),
        'SEGMT_FINANC_VEICULOS': buscar_valor_xml(root, 'SEGMT_FINANC/VL_FINANC_VEICL'),
        'SEGMT_FINANC_IMOB_EMPRESARIAL': buscar_valor_xml(root, 'SEGMT_FINANC/VL_FINANC_IMOBIL_EMPSRL'),
        'SEGMT_FINANC_IMOB_RESIDENCIAL': buscar_valor_xml(root, 'SEGMT_FINANC/VL_FINANC_IMOBIL_RESID'),
        'SEGMT_FINANC_OUTROS': buscar_valor_xml(root, 'SEGMT_FINANC/VL_FINANC_OUTRO'),

        # ─── Subsegmento: FACTORING ───
        'SEGMT_FACTORING_TOTAL': buscar_valor_xml(root, 'SEGMT_FACT/VL_SOM_SEGMT_FACT'),
        'SEGMT_FACTORING_PESSOA': buscar_valor_xml(root, 'SEGMT_FACT/VL_FACT_PESSOA'),
        'SEGMT_FACTORING_CORPORATIVO': buscar_valor_xml(root, 'SEGMT_FACT/VL_FACT_CORPOR'),

        # ─── Subsegmento: SETOR PÚBLICO ───
        'SEGMT_SETOR_PUBLICO_TOTAL': buscar_valor_xml(root, 'SEGMT_SETOR_PUBLIC/VL_SOM_SEGMT_SETOR_PUBLIC'),
        'SEGMT_PRECATORIOS': buscar_valor_xml(root, 'SEGMT_SETOR_PUBLIC/VL_SETOR_PUBLIC_PRECAT'),
        'SEGMT_CREDITOS_TRIBUTARIOS': buscar_valor_xml(root, 'SEGMT_SETOR_PUBLIC/VL_SETOR_PUBLIC_CRED_TRIBUT'),
        'SEGMT_ROYALTIES': buscar_valor_xml(root, 'SEGMT_SETOR_PUBLIC/VL_SETOR_PUBLIC_ROYA'),
        'SEGMT_SETOR_PUBLICO_OUTROS': buscar_valor_xml(root, 'SEGMT_SETOR_PUBLIC/VL_SETOR_PUBLIC_OUTRO'),

        # ═══ SEÇÃO 9: COMPOSIÇÃO DOS DICRED ADQUIRIDOS (COMPMT_DICRED_AQUIS) ═══
        'DICRED_AQUIS_PRAZO_VENC_TOTAL': buscar_valor_xml(root, 'COMPMT_DICRED_AQUIS/VL_SOM_PRAZO_VENC'),
        'DICRED_AQUIS_PRAZO_VENC_30': buscar_valor_xml(root, 'COMPMT_DICRED_AQUIS/VL_PRAZO_VENC_30'),
        'DICRED_AQUIS_PRAZO_VENC_31_60': buscar_valor_xml(root, 'COMPMT_DICRED_AQUIS/VL_PRAZO_VENC_31_60'),
        'DICRED_AQUIS_PRAZO_VENC_61_90': buscar_valor_xml(root, 'COMPMT_DICRED_AQUIS/VL_PRAZO_VENC_61_90'),
        'DICRED_AQUIS_PRAZO_VENC_91_120': buscar_valor_xml(root, 'COMPMT_DICRED_AQUIS/VL_PRAZO_VENC_91_120'),
        'DICRED_AQUIS_PRAZO_VENC_121_150': buscar_valor_xml(root, 'COMPMT_DICRED_AQUIS/VL_PRAZO_VENC_121_150'),
        'DICRED_AQUIS_PRAZO_VENC_151_180': buscar_valor_xml(root, 'COMPMT_DICRED_AQUIS/VL_PRAZO_VENC_151_180'),
        'DICRED_AQUIS_PRAZO_VENC_181_360': buscar_valor_xml(root, 'COMPMT_DICRED_AQUIS/VL_PRAZO_VENC_181_360'),
        'DICRED_AQUIS_PRAZO_VENC_361_720': buscar_valor_xml(root, 'COMPMT_DICRED_AQUIS/VL_PRAZO_VENC_361_720'),
        'DICRED_AQUIS_PRAZO_VENC_721_1080': buscar_valor_xml(root, 'COMPMT_DICRED_AQUIS/VL_PRAZO_VENC_721_1080'),
        'DICRED_AQUIS_PRAZO_VENC_ACIMA_1080': buscar_valor_xml(root, 'COMPMT_DICRED_AQUIS/VL_PRAZO_VENC_1080'),
        'DICRED_AQUIS_INAD_VENC_TOTAL': buscar_valor_xml(root, 'COMPMT_DICRED_AQUIS/VL_SOM_INAD_VENC'),
        'DICRED_AQUIS_INAD_VENC_30': buscar_valor_xml(root, 'COMPMT_DICRED_AQUIS/VL_INAD_VENC_30'),
        'DICRED_AQUIS_INAD_VENC_31_60': buscar_valor_xml(root, 'COMPMT_DICRED_AQUIS/VL_INAD_VENC_31_60'),
        'DICRED_AQUIS_INAD_VENC_61_90': buscar_valor_xml(root, 'COMPMT_DICRED_AQUIS/VL_INAD_VENC_61_90'),
        'DICRED_AQUIS_INAD_VENC_91_120': buscar_valor_xml(root, 'COMPMT_DICRED_AQUIS/VL_INAD_VENC_91_120'),
        'DICRED_AQUIS_INAD_VENC_121_150': buscar_valor_xml(root, 'COMPMT_DICRED_AQUIS/VL_INAD_VENC_121_150'),
        'DICRED_AQUIS_INAD_VENC_151_180': buscar_valor_xml(root, 'COMPMT_DICRED_AQUIS/VL_INAD_VENC_151_180'),
        'DICRED_AQUIS_INAD_VENC_181_360': buscar_valor_xml(root, 'COMPMT_DICRED_AQUIS/VL_INAD_VENC_181_360'),
        'DICRED_AQUIS_INAD_VENC_361_720': buscar_valor_xml(root, 'COMPMT_DICRED_AQUIS/VL_INAD_VENC_361_720'),
        'DICRED_AQUIS_INAD_VENC_721_1080': buscar_valor_xml(root, 'COMPMT_DICRED_AQUIS/VL_INAD_VENC_721_1080'),
        'DICRED_AQUIS_INAD_VENC_ACIMA_1080': buscar_valor_xml(root, 'COMPMT_DICRED_AQUIS/VL_INAD_VENC_1080'),
        'DICRED_AQUIS_PAGO_ANTCP_TOTAL': buscar_valor_xml(root, 'COMPMT_DICRED_AQUIS/VL_SOM_PAGO_ANTCP'),
        'DICRED_AQUIS_PAGO_ANTCP_30': buscar_valor_xml(root, 'COMPMT_DICRED_AQUIS/VL_PAGO_ANTCP_30'),
        'DICRED_AQUIS_PAGO_ANTCP_31_60': buscar_valor_xml(root, 'COMPMT_DICRED_AQUIS/VL_PAGO_ANTCP_31_60'),
        'DICRED_AQUIS_PAGO_ANTCP_61_90': buscar_valor_xml(root, 'COMPMT_DICRED_AQUIS/VL_PAGO_ANTCP_61_90'),
        'DICRED_AQUIS_PAGO_ANTCP_91_120': buscar_valor_xml(root, 'COMPMT_DICRED_AQUIS/VL_PAGO_ANTCP_91_120'),
        'DICRED_AQUIS_PAGO_ANTCP_121_150': buscar_valor_xml(root, 'COMPMT_DICRED_AQUIS/VL_PAGO_ANTCP_121_150'),
        'DICRED_AQUIS_PAGO_ANTCP_151_180': buscar_valor_xml(root, 'COMPMT_DICRED_AQUIS/VL_PAGO_ANTCP_151_180'),
        'DICRED_AQUIS_PAGO_ANTCP_181_360': buscar_valor_xml(root, 'COMPMT_DICRED_AQUIS/VL_PAGO_ANTCP_181_360'),
        'DICRED_AQUIS_PAGO_ANTCP_361_720': buscar_valor_xml(root, 'COMPMT_DICRED_AQUIS/VL_PAGO_ANTCP_361_720'),
        'DICRED_AQUIS_PAGO_ANTCP_721_1080': buscar_valor_xml(root, 'COMPMT_DICRED_AQUIS/VL_PAGO_ANTCP_721_1080'),
        'DICRED_AQUIS_PAGO_ANTCP_ACIMA_1080': buscar_valor_xml(root, 'COMPMT_DICRED_AQUIS/VL_PAGO_ANTCP_1080'),

        # ═══ SEÇÃO 10: COMPOSIÇÃO DOS DICRED SEM AQUISIÇÃO (COMPMT_DICRED_SEM_AQUIS) ═══
        'DICRED_SEM_AQUIS_PRAZO_VENC_TOTAL': buscar_valor_xml(root, 'COMPMT_DICRED_SEM_AQUIS/VL_SOM_PRAZO_VENC'),
        'DICRED_SEM_AQUIS_PRAZO_VENC_30': buscar_valor_xml(root, 'COMPMT_DICRED_SEM_AQUIS/VL_PRAZO_VENC_30'),
        'DICRED_SEM_AQUIS_PRAZO_VENC_31_60': buscar_valor_xml(root, 'COMPMT_DICRED_SEM_AQUIS/VL_PRAZO_VENC_31_60'),
        'DICRED_SEM_AQUIS_PRAZO_VENC_61_90': buscar_valor_xml(root, 'COMPMT_DICRED_SEM_AQUIS/VL_PRAZO_VENC_61_90'),
        'DICRED_SEM_AQUIS_PRAZO_VENC_91_120': buscar_valor_xml(root, 'COMPMT_DICRED_SEM_AQUIS/VL_PRAZO_VENC_91_120'),
        'DICRED_SEM_AQUIS_PRAZO_VENC_121_150': buscar_valor_xml(root, 'COMPMT_DICRED_SEM_AQUIS/VL_PRAZO_VENC_121_150'),
        'DICRED_SEM_AQUIS_PRAZO_VENC_151_180': buscar_valor_xml(root, 'COMPMT_DICRED_SEM_AQUIS/VL_PRAZO_VENC_151_180'),
        'DICRED_SEM_AQUIS_PRAZO_VENC_181_360': buscar_valor_xml(root, 'COMPMT_DICRED_SEM_AQUIS/VL_PRAZO_VENC_181_360'),
        'DICRED_SEM_AQUIS_PRAZO_VENC_361_720': buscar_valor_xml(root, 'COMPMT_DICRED_SEM_AQUIS/VL_PRAZO_VENC_361_720'),
        'DICRED_SEM_AQUIS_PRAZO_VENC_721_1080': buscar_valor_xml(root, 'COMPMT_DICRED_SEM_AQUIS/VL_PRAZO_VENC_721_1080'),
        'DICRED_SEM_AQUIS_PRAZO_VENC_ACIMA_1080': buscar_valor_xml(root, 'COMPMT_DICRED_SEM_AQUIS/VL_PRAZO_VENC_1080'),
        'DICRED_SEM_AQUIS_INAD_VENC_TOTAL': buscar_valor_xml(root, 'COMPMT_DICRED_SEM_AQUIS/VL_SOM_INAD_VENC'),
        'DICRED_SEM_AQUIS_INAD_VENC_30': buscar_valor_xml(root, 'COMPMT_DICRED_SEM_AQUIS/VL_INAD_VENC_30'),
        'DICRED_SEM_AQUIS_INAD_VENC_31_60': buscar_valor_xml(root, 'COMPMT_DICRED_SEM_AQUIS/VL_INAD_VENC_31_60'),
        'DICRED_SEM_AQUIS_INAD_VENC_61_90': buscar_valor_xml(root, 'COMPMT_DICRED_SEM_AQUIS/VL_INAD_VENC_61_90'),
        'DICRED_SEM_AQUIS_INAD_VENC_91_120': buscar_valor_xml(root, 'COMPMT_DICRED_SEM_AQUIS/VL_INAD_VENC_91_120'),
        'DICRED_SEM_AQUIS_INAD_VENC_121_150': buscar_valor_xml(root, 'COMPMT_DICRED_SEM_AQUIS/VL_INAD_VENC_121_150'),
        'DICRED_SEM_AQUIS_INAD_VENC_151_180': buscar_valor_xml(root, 'COMPMT_DICRED_SEM_AQUIS/VL_INAD_VENC_151_180'),
        'DICRED_SEM_AQUIS_INAD_VENC_181_360': buscar_valor_xml(root, 'COMPMT_DICRED_SEM_AQUIS/VL_INAD_VENC_181_360'),
        'DICRED_SEM_AQUIS_INAD_VENC_361_720': buscar_valor_xml(root, 'COMPMT_DICRED_SEM_AQUIS/VL_INAD_VENC_361_720'),
        'DICRED_SEM_AQUIS_INAD_VENC_721_1080': buscar_valor_xml(root, 'COMPMT_DICRED_SEM_AQUIS/VL_INAD_VENC_721_1080'),
        'DICRED_SEM_AQUIS_INAD_VENC_ACIMA_1080': buscar_valor_xml(root, 'COMPMT_DICRED_SEM_AQUIS/VL_INAD_VENC_1080'),
        'DICRED_SEM_AQUIS_PAGO_ANTCP_TOTAL': buscar_valor_xml(root, 'COMPMT_DICRED_SEM_AQUIS/VL_SOM_PAGO_ANTCP'),
        'DICRED_SEM_AQUIS_PAGO_ANTCP_30': buscar_valor_xml(root, 'COMPMT_DICRED_SEM_AQUIS/VL_PAGO_ANTCP_30'),
        'DICRED_SEM_AQUIS_PAGO_ANTCP_31_60': buscar_valor_xml(root, 'COMPMT_DICRED_SEM_AQUIS/VL_PAGO_ANTCP_31_60'),
        'DICRED_SEM_AQUIS_PAGO_ANTCP_61_90': buscar_valor_xml(root, 'COMPMT_DICRED_SEM_AQUIS/VL_PAGO_ANTCP_61_90'),
        'DICRED_SEM_AQUIS_PAGO_ANTCP_91_120': buscar_valor_xml(root, 'COMPMT_DICRED_SEM_AQUIS/VL_PAGO_ANTCP_91_120'),
        'DICRED_SEM_AQUIS_PAGO_ANTCP_121_150': buscar_valor_xml(root, 'COMPMT_DICRED_SEM_AQUIS/VL_PAGO_ANTCP_121_150'),
        'DICRED_SEM_AQUIS_PAGO_ANTCP_151_180': buscar_valor_xml(root, 'COMPMT_DICRED_SEM_AQUIS/VL_PAGO_ANTCP_151_180'),
        'DICRED_SEM_AQUIS_PAGO_ANTCP_181_360': buscar_valor_xml(root, 'COMPMT_DICRED_SEM_AQUIS/VL_PAGO_ANTCP_181_360'),
        'DICRED_SEM_AQUIS_PAGO_ANTCP_361_720': buscar_valor_xml(root, 'COMPMT_DICRED_SEM_AQUIS/VL_PAGO_ANTCP_361_720'),
        'DICRED_SEM_AQUIS_PAGO_ANTCP_721_1080': buscar_valor_xml(root, 'COMPMT_DICRED_SEM_AQUIS/VL_PAGO_ANTCP_721_1080'),
        'DICRED_SEM_AQUIS_PAGO_ANTCP_ACIMA_1080': buscar_valor_xml(root, 'COMPMT_DICRED_SEM_AQUIS/VL_PAGO_ANTCP_1080')
    }

    # ─────────────────────────────────────────────────────────────────────────
    # CÁLCULO DE INDICADORES CRÍTICOS
    # ─────────────────────────────────────────────────────────────────────────

    # Inadimplência consolidada (prioriza maior valor entre CRED e DICRED)
    inadimpl_cred = dados['CRED_INADIMPLENCIA']
    inadimpl_dicred = dados['DICRED_INADIMPLENCIA']
    dados['INADIMPLENCIA_TOTAL'] = max(inadimpl_cred, inadimpl_dicred)

    # Carteira de crédito para cálculo de NPL
    carteira_credito = dados['CREDITOS_ADQUIRIDOS']

    # ### NOVO: Verifica se 'DATA_COMPETENCIA' é uma string válida para conversão
    if isinstance(dados['DATA_COMPETENCIA'], str):
        try:
            # Converte para formato YYYY-MM-DD para garantir ordenação correta
            # Assume que a data está no formato MM/YYYY
            month, year = map(int, dados['DATA_COMPETENCIA'].split('/'))
            dados['DATA_COMPETENCIA_ORDENAVEL'] = f"{year:04d}-{month:02d}-01"
        except (ValueError, AttributeError):
            dados['DATA_COMPETENCIA_ORDENAVEL'] = None # Ou manter o valor original
    else:
        dados['DATA_COMPETENCIA_ORDENAVEL'] = None

    # ###

    # Índice de NPL (Non-Performing Loans) - CORRIGIDO
    if carteira_credito > 0 and dados['INADIMPLENCIA_TOTAL'] > 0:
        dados['INDICE_NPL_PERCENTUAL'] = (dados['INADIMPLENCIA_TOTAL'] / carteira_credito) * 100
    else:
        dados['INDICE_NPL_PERCENTUAL'] = 0.0

    # Taxa de liquidez imediata (Disponibilidades / Ativo Total)
    if dados['ATIVO_TOTAL'] > 0:
        dados['TAXA_LIQUIDEZ_PERCENTUAL'] = (dados['DISPONIBILIDADES'] / dados['ATIVO_TOTAL']) * 100
    else:
        dados['TAXA_LIQUIDEZ_PERCENTUAL'] = 0.0

    # Concentração em crédito (Carteira / Ativo Total)
    if dados['ATIVO_TOTAL'] > 0:
        dados['CONCENTRACAO_CREDITO_PERCENTUAL'] = (carteira_credito / dados['ATIVO_TOTAL']) * 100
    else:
        dados['CONCENTRACAO_CREDITO_PERCENTUAL'] = 0.0

    return dados


print("✅ Função de extração completa definida:")
print("   • 90+ campos extraídos")
print("   • 4 indicadores calculados automaticamente")
print("   • Tratamento robusto de valores ausentes")

# ═════════════════════════════════════════════════════════════════════
# CÉLULA 4: FUNÇÃO ETL COMPLETA
# ═════════════════════════════════════════════════════════════════════

def etl_fidc_completo(cnpj_alvo: str, nome_fundo_referencia: str = "") -> List[Dict]:
    """
    Executa o pipeline ETL completo para um CNPJ, lidando com múltiplos fundos
    associados e extraindo o informe mensal mais recente para cada um.

    Workflow:
        1. Discovery: Busca documentos do fundo na API B3 com limite maior.
        2. Filter: Filtra estritamente por 'Informe Mensal Estruturado'.
        3. Deduplicate: Seleciona o informe mais recente para cada fundo único.
        4. Download: Baixa o XML de cada informe selecionado.
        5. Parse & Extract: Decodifica e processa o XML, extraindo 90+ campos.
        6. Cleanup: Remove arquivos temporários.

    Args:
        cnpj_alvo: CNPJ do fundo (string com 14 dígitos)
        nome_fundo_referencia: Nome do fundo (opcional, para logs e possível filtragem inicial)

    Returns:
        Uma lista de dicionários, onde cada dicionário contém os dados extraídos
        de um fundo único para o CNPJ alvo. Retorna uma lista vazia se nenhum
        documento válido for encontrado ou em caso de erro.
    """

    resultados_fundo_cnpj = []

    try:
        # ─────────────────────────────────────────────────────────────────────
        # ETAPA 1: DISCOVERY - Buscar documentos disponíveis
        # ─────────────────────────────────────────────────────────────────────

        params = {
            'd': 0,         # Documento
            's': 0,         # Start (paginação)
            'l': 200,       # Limit (aumentado para 200)
            'cnpjFundo': cnpj_alvo
        }

        resp_busca = requests.get(
            URL_API_BUSCA,
            params=params,
            headers=HEADERS,
            timeout=TIMEOUT_BUSCA
        )
        resp_busca.raise_for_status()

        data = resp_busca.json().get('data', [])

        if not data:
            # print(f"DEBUG: etl_fidc_completo para {cnpj_alvo}: NENHUM DADO DA API. Retornando lista vazia.")
            return []

        df_docs = pd.DataFrame(data)

        # ─────────────────────────────────────────────────────────────────────
        # ETAPA 2: FILTER - Filtrar estritamente por 'Informe Mensal Estruturado'
        # ─────────────────────────────────────────────────────────────────────

        # Filtrar apenas 'Informe Mensal Estruturado' e status 'Ativo'
        # Adicionado .str.strip() para lidar com espaços em branco inesperados
        df_mensal = df_docs[
            (df_docs['tipoDocumento'].str.strip() == "Informe Mensal Estruturado") &
            (df_docs['situacaoDocumento'].str.strip() == "A")
        ].copy()

        if df_mensal.empty:
            # print(f"DEBUG: etl_fidc_completo para {cnpj_alvo}: df_mensal vazio após filtro tipo/status. Retornando lista vazia.")
            return []

        # ─────────────────────────────────────────────────────────────────────
        # ETAPA 3: DEDUPLICATE - Selecionar o mais recente para cada fundo único
        # ─────────────────────────────────────────────────────────────────────

        # Converte 'dataReferencia' para um formato ordenável (MM/YYYY -> YYYY-MM)
        # Adiciona '01' para criar uma data completa, facilitando a conversão para datetime
        df_mensal['dataReferenciaOrdenavel'] = pd.to_datetime(
            df_mensal['dataReferencia'].str[3:] + '-' + df_mensal['dataReferencia'].str[:2] + '-01',
            format='%Y-%m-%d', errors='coerce'
        )

        # Remove linhas com dataReferencia inválida (NaT)
        df_mensal.dropna(subset=['dataReferenciaOrdenavel'], inplace=True)

        if df_mensal.empty:
            # print(f"DEBUG: etl_fidc_completo para {cnpj_alvo}: df_mensal vazio após dropna de datas. Retornando lista vazia.")
            return []

        # DEBUG PRINT: Mostrar df_mensal antes da deduplicação para o CNPJ de interesse
        # if cnpj_alvo == '51199121000145':
            # print(f"DEBUG: df_mensal para {cnpj_alvo} antes da deduplicação:\n{df_mensal[['id', 'dataReferencia', 'dataReferenciaOrdenavel', 'descricaoFundo']].to_string()}")


        # Ordena por nome do fundo e data de referência (mais recente primeiro)
        df_mensal.sort_values(by=['descricaoFundo', 'dataReferenciaOrdenavel'], ascending=[True, False], inplace=True)

        # Pega o documento mais recente para cada fundo único (baseado em descricaoFundo)
        df_fundos_unicos = df_mensal.groupby('descricaoFundo').first().reset_index()
        # print(f"DEBUG: etl_fidc_completo para {cnpj_alvo}: Encontrado(s) {len(df_fundos_unicos)} fundo(s) único(s) após deduplicação.")

        # Itera sobre cada fundo único encontrado
        for index, doc_info in df_fundos_unicos.iterrows():
            doc_id = doc_info['id']
            data_referencia = doc_info.get('dataReferencia', 'N/A')
            nome_fundo_doc = doc_info.get('descricaoFundo', 'Nome N/A')

            # ────────────────────────────────────────────────────────────────────
            # ETAPA 4: DOWNLOAD - Baixar XML do documento
            # ────────────────────────────────────────────────────────────────────
            nome_arquivo_temp = None # Reinicia para cada iteração
            try:
                url_download = f"{URL_API_DOWNLOAD}?id={doc_id}"
                resp_download = requests.get(
                    url_download,
                    headers=HEADERS,
                    timeout=TIMEOUT_DOWNLOAD
                )
                resp_download.raise_for_status()

                # Decodificar XML (vem em Base64)
                xml_content = base64.b64decode(resp_download.content)

                # Salvar temporariamente
                nome_arquivo_temp = f'temp_{cnpj_alvo}_{doc_id}.xml'
                with open(nome_arquivo_temp, 'wb') as f:
                    f.write(xml_content)

                # ─────────────────────────────────────────────────────────────────
                # ETAPA 5: EXTRACT - Extrair todos os dados do XML
                # ─────────────────────────────────────────────────────────────────

                dados_extraidos = extrair_dados_xml_completo(xml_content)

                # Garante que CNPJ_FUNDO usa cnpj_alvo como fallback
                if not dados_extraidos.get('CNPJ_FUNDO'):
                    dados_extraidos['CNPJ_FUNDO'] = cnpj_alvo

                # Adicionar metadados do processo
                dados_extraidos['NOME_FUNDO'] = nome_fundo_doc # Usa o nome do fundo do documento
                dados_extraidos['STATUS'] = 'SUCESSO'
                dados_extraidos['ID_DOCUMENTO'] = doc_id
                dados_extraidos['DATA_REFERENCIA_DOC'] = data_referencia
                dados_extraidos['MENSAGEM_ERRO'] = None

                resultados_fundo_cnpj.append(dados_extraidos)

            except requests.exceptions.HTTPError as e:
                resultados_fundo_cnpj.append({
                    'CNPJ_FUNDO': cnpj_alvo,
                    'NOME_FUNDO': nome_fundo_doc,
                    'STATUS': 'ERRO_HTTP_DOWNLOAD',
                    'MENSAGEM_ERRO': f'HTTP Error no download do doc {doc_id}: {str(e)}'
                })
            except requests.exceptions.Timeout as e:
                resultados_fundo_cnpj.append({
                    'CNPJ_FUNDO': cnpj_alvo,
                    'NOME_FUNDO': nome_fundo_doc,
                    'STATUS': 'TIMEOUT_DOWNLOAD',
                    'MENSAGEM_ERRO': f'Timeout no download do doc {doc_id}: {str(e)}'
                })
            except requests.exceptions.RequestException as e:
                resultados_fundo_cnpj.append({
                    'CNPJ_FUNDO': cnpj_alvo,
                    'NOME_FUNDO': nome_fundo_doc,
                    'STATUS': 'ERRO_CONEXAO_DOWNLOAD',
                    'MENSAGEM_ERRO': f"Erro de conexão no download do doc {doc_id}: {str(e)}"
                })
            except ET.ParseError as e:
                resultados_fundo_cnpj.append({
                    'CNPJ_FUNDO': cnpj_alvo,
                    'NOME_FUNDO': nome_fundo_doc,
                    'STATUS': 'ERRO_PARSE_XML',
                    'MENSAGEM_ERRO': f'Erro ao processar XML do doc {doc_id}: {str(e)}'
                })
            except Exception as e:
                resultados_fundo_cnpj.append({
                    'CNPJ_FUNDO': cnpj_alvo,
                    'NOME_FUNDO': nome_fundo_doc,
                    'STATUS': 'ERRO_INESPERADO_DOC',
                    'MENSAGEM_ERRO': f'Erro inesperado ao processar doc {doc_id}: {str(e)}'
                })
            finally:
                # ─────────────────────────────────────────────────────────────────
                # ETAPA 6: CLEANUP - Remover arquivo temporário
                # ─────────────────────────────────────────────────────────────────
                if nome_arquivo_temp and os.path.exists(nome_arquivo_temp):
                    try:
                        os.remove(nome_arquivo_temp)
                    except OSError:
                        pass  # Ignora erro na remoção

    except requests.exceptions.HTTPError as e:
        resultados_fundo_cnpj.append({
            'CNPJ_FUNDO': cnpj_alvo,
            'NOME_FUNDO': nome_fundo_referencia,
            'STATUS': 'ERRO_HTTP_BUSCA',
            'MENSAGEM_ERRO': f'HTTP Error na busca de documentos: {str(e)}'
        })
    except requests.exceptions.Timeout as e:
        resultados_fundo_cnpj.append({
            'CNPJ_FUNDO': cnpj_alvo,
            'NOME_FUNDO': nome_fundo_referencia,
            'STATUS': 'TIMEOUT_BUSCA',
            'MENSAGEM_ERRO': f'Timeout na busca de documentos: {str(e)}'
        })
    except requests.exceptions.RequestException as e:
        resultados_fundo_cnpj.append({
            'CNPJ_FUNDO': cnpj_alvo,
            'NOME_FUNDO': nome_fundo_referencia,
            'STATUS': 'ERRO_CONEXAO_BUSCA',
            'MENSAGEM_ERRO': f"Erro de conexão na busca de documentos: {str(e)}"
        })
    except Exception as e:
        resultados_fundo_cnpj.append({
            'CNPJ_FUNDO': cnpj_alvo,
            'NOME_FUNDO': nome_fundo_referencia,
            'STATUS': 'ERRO_INESPERADO_GERAL',
            'MENSAGEM_ERRO': f'Erro inesperado no processo ETL para {cnpj_alvo}: {str(e)}'
        })

    return resultados_fundo_cnpj


# REMOVIDO: print("✅ Função ETL completa definida!") para evitar poluir o output durante debug
# REMOVIDO: print("   • Discovery → Download → Parse → Extract → Cleanup")
# REMOVIDO: print("   • Tratamento completo de erros")
# REMOVIDO: print("   • Cleanup automático garantido")
# REMOVIDO: print("   • Suporte a múltiplos fundos por CNPJ com último informe mensal")

# ═════════════════════════════════════════════════════════════════════
# CÉLULA 5: CARREGAMENTO DOS CNPJs DO CSV
# ═════════════════════════════════════════════════════════════════════

# Nome do arquivo CSV de entrada (faça upload deste arquivo no Colab)
ARQUIVO_CSV_ENTRADA = 'lista_cnpjs_fidc.csv'

try:
    # Carregar CSV com lista de CNPJs
    df_cnpjs_entrada = pd.read_csv(ARQUIVO_CSV_ENTRADA, encoding='utf-8-sig')

    print(f"✅ Arquivo '{ARQUIVO_CSV_ENTRADA}' carregado com sucesso!")
    print(f"   • Total de CNPJs: {len(df_cnpjs_entrada)}")

    # Converter CNPJ para string (remove notação científica)
    df_cnpjs_entrada['CNPJ'] = df_cnpjs_entrada['CNPJ'].astype(str).str.replace('.0', '', regex=False)

    # Garantir que CNPJ tem 14 dígitos
    df_cnpjs_entrada['CNPJ'] = df_cnpjs_entrada['CNPJ'].str.zfill(14)

    print(f"\n📋 Primeiras 5 linhas:")
    display(df_cnpjs_entrada.head())

    print(f"\n⏱️  Tempo estimado de processamento:")
    tempo_estimado = len(df_cnpjs_entrada) * DELAY_ENTRE_REQUISICOES / 60
    print(f"   • {tempo_estimado:.1f} minutos (com delay de {DELAY_ENTRE_REQUISICOES}s)")

except FileNotFoundError:
    print(f"❌ ERRO: Arquivo '{ARQUIVO_CSV_ENTRADA}' não encontrado!")
    print(f"\n📝 Instruções:")
    print(f"   1. Faça upload do arquivo '{ARQUIVO_CSV_ENTRADA}' no Colab")
    print(f"   2. Execute esta célula novamente")
    print(f"\nPara fazer upload: Use o ícone de pasta (📁) na barra lateral esquerda")
    raise

# ═════════════════════════════════════════════════════════════════════
# CÉLULA 6: PROCESSAMENTO EM LOTE
# ═════════════════════════════════════════════════════════════════════

print(f"\n{'═'*80}")
print(f"🚀 INICIANDO PROCESSAMENTO DE {len(df_cnpjs_entrada)} CNPJs")
print(f"{'═'*80}\n")

resultados_consolidados = []

for i, row in df_cnpjs_entrada.iterrows():
    cnpj = row['CNPJ']
    nome_fundo_ref = row.get('NOME_FUNDO', '') # Usar o nome do CSV como referência

    # Truncar nome para log (máx 50 caracteres)
    nome_display = nome_fundo_ref[:47] + "..." if len(nome_fundo_ref) > 50 else nome_fundo_ref

    print(f"[{i+1}/{len(df_cnpjs_entrada)}] {cnpj} - {nome_display}... ", end="")

    # Executar ETL para cada CNPJ, que agora pode retornar uma lista de resultados
    resultados_atuais = etl_fidc_completo(cnpj, nome_fundo_ref)

    if resultados_atuais:
        resultados_consolidados.extend(resultados_atuais)
        # Log do primeiro resultado para simplicidade no feedback
        primeiro_resultado = resultados_atuais[0]
        if primeiro_resultado.get('STATUS') == 'SUCESSO':
            data_ref = primeiro_resultado.get('DATA_REFERENCIA_DOC', 'N/A')
            npl = primeiro_resultado.get('INDICE_NPL_PERCENTUAL', 0)
            num_fundos = len(resultados_atuais)
            print(f"✅ OK ({num_fundos} Fundo(s) - Ref: {data_ref}, NPL: {npl:.2f}%) ")
        else:
            status = primeiro_resultado.get('STATUS', 'ERRO')
            # Inclui o nome do fundo para facilitar a identificação do erro
            print(f"❌ {status} (Erro para '{primeiro_resultado.get('NOME_FUNDO', 'N/A')}')")
    else:
        # Caso a função retorne lista vazia ou erro geral na busca
        resultados_consolidados.append({
            'CNPJ_FUNDO': cnpj,
            'NOME_FUNDO': nome_fundo_ref,
            'STATUS': 'NENHUM_INFORME_VALIDO',
            'MENSAGEM_ERRO': 'Nenhum informe mensal estruturado e ativo encontrado para este CNPJ.'
        })
        print(f"❌ NENHUM_INFORME_VALIDO")

    # Delay entre requisições (evita bloqueio da API)
    if i < len(df_cnpjs_entrada) - 1:  # Não espera no último
        time.sleep(DELAY_ENTRE_REQUISICOES)

    # Checkpoint a cada 50 CNPJs (salva progresso intermediário)
    if (i + 1) % 50 == 0:
        df_temp = pd.DataFrame(resultados_consolidados)
        df_temp.to_csv('checkpoint_temp.csv', index=False, encoding='utf-8-sig', sep=';', decimal=',')
        print(f"   💾 Checkpoint salvo ({i+1} CNPJs processados)")

print(f"\n{'═'*80}")
print(f"✅ PROCESSAMENTO CONCLUÍDO!")
print(f"{'═'*80}\n")

# ═════════════════════════════════════════════════════════════════════
# CÉLULA 7: CONSOLIDAÇÃO E ESTATÍSTICAS
# ═════════════════════════════════════════════════════════════════════

# Criar DataFrame final
df_resultado_final = pd.DataFrame(resultados_consolidados)

# Estatísticas de processamento
total_sucesso = len(df_resultado_final[df_resultado_final['STATUS'] == 'SUCESSO'])
total_erros = len(df_resultado_final[df_resultado_final['STATUS'] != 'SUCESSO'])

print(f"📊 ESTATÍSTICAS DO PROCESSAMENTO")
print(f"{'─'*80}")
print(f"   Total de CNPJs de entrada: {len(df_cnpjs_entrada)}")
print(f"   Total de Informes processados (incluindo erros): {len(df_resultado_final)}")
print(f"   ✅ Sucesso: {total_sucesso} ({total_sucesso/len(df_resultado_final)*100:.1f}%) [Fundos únicos com informes válidos]")
print(f"   ❌ Erros: {total_erros} ({total_erros/len(df_resultado_final)*100:.1f}%) [Informações de erro geradas]")

# Distribuição de erros
if total_erros > 0:
    print(f"\n⚠️  TIPOS DE ERROS:")
    erros_por_tipo = df_resultado_final[df_resultado_final['STATUS'] != 'SUCESSO']['STATUS'].value_counts()
    for status, count in erros_por_tipo.items():
        # Aqui também adicionamos uma verificação para evitar IndexError se houver apenas um erro de um tipo específico.
        # E garantimos que a mensagem de erro seja acessada de forma segura.
        error_message_sample = df_resultado_final[df_resultado_final['STATUS'] == status]['MENSAGEM_ERRO'].iloc[0] if not df_resultado_final[df_resultado_final['STATUS'] == status].empty else 'N/A'
        print(f"   • {status}: {count} ({count/total_erros*100:.1f}%) - '{error_message_sample}'")

# Estatísticas dos dados extraídos (apenas sucessos)
df_sucesso = df_resultado_final[df_resultado_final['STATUS'] == 'SUCESSO'].copy()

if not df_sucesso.empty:
    print(f"\n💰 ESTATÍSTICAS FINANCEIRAS (Fundos com sucesso)")
    print(f"{'─'*80}")

    ativo_total_soma = df_sucesso['ATIVO_TOTAL'].sum()
    ativo_total_media = df_sucesso['ATIVO_TOTAL'].mean()
    ativo_total_mediana = df_sucesso['ATIVO_TOTAL'].median()

    inadimpl_soma = df_sucesso['INADIMPLENCIA_TOTAL'].sum()
    npl_medio = df_sucesso['INDICE_NPL_PERCENTUAL'].mean()
    npl_mediano = df_sucesso['INDICE_NPL_PERCENTUAL'].median()
    npl_max = df_sucesso['INDICE_NPL_PERCENTUAL'].max()

    print(f"   Ativo Total (Soma):           R$ {ativo_total_soma:,.2f}")
    print(f"   Ativo Total (Média):          R$ {ativo_total_media:,.2f}")
    print(f"   Ativo Total (Mediana):        R$ {ativo_total_mediana:,.2f}")
    print(f"   Inadimplência (Soma):         R$ {inadimpl_soma:,.2f}")
    print(f"   NPL Médio:                    {npl_medio:.2f}%")
    print(f"   NPL Mediano:                  {npl_mediano:.2f}%")
    print(f"   NPL Máximo:                   {npl_max:.2f}%")

    # Fundos com maior NPL
    df_npl_alto = df_sucesso[df_sucesso['INDICE_NPL_PERCENTUAL'] > 5.0].sort_values('INDICE_NPL_PERCENTUAL', ascending=False)

    if not df_npl_alto.empty:
        print(f"\n⚠️  TOP 10 FUNDOS COM MAIOR NPL (> 5%)")
        print(f"{'─'*80}")
        for idx, row in df_npl_alto.head(10).iterrows():
            nome = row['NOME_FUNDO'][:45] + "..." if len(row['NOME_FUNDO']) > 48 else row['NOME_FUNDO']
            cnpj = row['CNPJ_FUNDO']
            npl = row['INDICE_NPL_PERCENTUAL']
            ativo = row['ATIVO_TOTAL']
            print(f"   • {nome}")
            print(f"     CNPJ: {cnpj} | NPL: {npl:.2f}% | Ativo: R$ {ativo:,.2f}")

# Preview do DataFrame
print(f"\n📋 PREVIEW DOS DADOS (Primeiras 10 linhas)")
print(f"{'─'*80}")
colunas_preview = ['CNPJ_FUNDO', 'NOME_FUNDO', 'STATUS', 'DATA_REFERENCIA_DOC',
                   'ATIVO_TOTAL', 'INADIMPLENCIA_TOTAL', 'INDICE_NPL_PERCENTUAL']
display(df_resultado_final[colunas_preview].head(10))

# ═════════════════════════════════════════════════════════════════════
# CÉLULA 8: EXPORTAÇÃO DOS DATOS
# ═════════════════════════════════════════════════════════════════════

# Exportar para CSV (formato universal)
nome_arquivo_csv = 'base_fidc_completa.csv'
df_resultado_final.to_csv(nome_arquivo_csv, index=False, encoding='utf-8-sig', sep=';', decimal=',')

print(f"\n💾 EXPORTAÇÃO DOS DATOS")
print(f"{'─'*80}")
print(f"   ✅ CSV salvo: {nome_arquivo_csv}")
print(f"      • Encoding: UTF-8 com BOM (compatível com Excel)")
print(f"      • Separador: ponto-e-vírgula (;)")
print(f"      • Decimal: vírgula (,)")
print(f"      • Linhas: {len(df_resultado_final)}")
print(f"      • Colunas: {len(df_resultado_final.columns)}")

# Exportar apenas fundos com sucesso (opcional)
if total_sucesso > 0:
    nome_arquivo_sucesso = 'base_fidc_sucesso.csv'
    df_sucesso.to_csv(nome_arquivo_sucesso, index=False, encoding='utf-8-sig', sep=';', decimal=',')
    print(f"   ✅ CSV (só sucessos): {nome_arquivo_sucesso}")
    print(f"      • Linhas: {len(df_sucesso)}")

# Exportar fundos com erro (para análise)
if total_erros > 0:
    df_erros = df_resultado_final[df_resultado_final['STATUS'] != 'SUCESSO']
    nome_arquivo_erros = 'base_fidc_erros.csv'
    df_erros.to_csv(nome_arquivo_erros, index=False, encoding='utf-8-sig', sep=';', decimal=',')
    print(f"   ⚠️  CSV (erros): {nome_arquivo_erros}")
    print(f"      • Linhas: {len(df_erros)}")

# Exportar para Excel (se necessário)
try:
    nome_arquivo_excel = 'base_fidc_completa.xlsx'

    # Criar Excel com múltiplas abas
    with pd.ExcelWriter(nome_arquivo_excel, engine='openpyxl') as writer:
        df_resultado_final.to_excel(writer, sheet_name='Todos', index=False)

        if total_sucesso > 0:
            df_sucesso.to_excel(writer, sheet_name='Sucesso', index=False)

        if total_erros > 0:
            df_erros.to_excel(writer, sheet_name='Erros', index=False)

    print(f"   ✅ Excel salvo: {nome_arquivo_excel}")
    print(f"      • Abas: Todos ({len(df_resultado_final)}), Sucesso ({total_sucesso}), Erros ({total_erros})")

except ImportError:
    print(f"   ⚠️  Excel não gerado (instale: !pip install openpyxl)")

print(f"\n{'═'*80}")
print(f"🎉 ETL CONCLUÍDO COM SUCESSO!")
print(f"{'═'*80}")
print(f"\n📥 Para baixar os arquivos:")
print(f"   1. Clique no ícone de pasta (📁) na barra lateral esquerda")
print(f"   2. Localize os arquivos CSV/Excel gerados")
print(f"   3. Clique nos 3 pontinhos (...) ao lado do arquivo")
print(f"   4. Selecione 'Download'")
print(f"\n🔄 Próximos passos:")
print(f"   1. Importe '{nome_arquivo_csv}' no Power BI / Tableau / Excel")
print(f"   2. Crie dashboards com os 90+ campos disponíveis")
print(f"   3. Analise fundos com alto NPL")
print(f"   4. Identifique tendências de inadimplência por segmento")

✅ Bibliotecas importadas com sucesso!
📊 Configurações:
   • Delay entre requisições: 2s
   • Timeout busca: 10s
   • Timeout download: 20s
✅ Funções auxiliares definidas:
   • limpar_tag()
   • converter_valor()
   • buscar_valor_xml()
✅ Função de extração completa definida:
   • 90+ campos extraídos
   • 4 indicadores calculados automaticamente
   • Tratamento robusto de valores ausentes
✅ Arquivo 'lista_cnpjs_fidc.csv' carregado com sucesso!
   • Total de CNPJs: 441

📋 Primeiras 5 linhas:


,CNPJ,NOME_FUNDO,CNPJ_ORIGINAL
0,51199121000145,2MONEY RESP LIMITADA FIDC NP SUBORDINADA JÚNIOR 1,51.199.121/0001-45
1,47388724000118,3R RESP LIMITADA FIDC NP ÚNICA 1,47.388.724/0001-18
2,52362629000185,3T FICFIDC NP 1,52.362.629/0001-85
3,50356663000111,ABF VISION FIDC NP SÊNIOR 1,50.356.663/0001-11
4,41609997000169,ACERTO GRUPO INTER FIDC NP SUBORDINADA JÚNIOR 1,41.609.997/0001-69



⏱️  Tempo estimado de processamento:
   • 14.7 minutos (com delay de 2s)

════════════════════════════════════════════════════════════════════════════════
🚀 INICIANDO PROCESSAMENTO DE 441 CNPJs
════════════════════════════════════════════════════════════════════════════════

[1/441] 51199121000145 - 2MONEY RESP LIMITADA FIDC NP SUBORDINADA JÚNIOR 1... ✅ OK (1 Fundo(s) - Ref: 11/2025, NPL: 0.00%) 
[2/441] 47388724000118 - 3R RESP LIMITADA FIDC NP ÚNICA 1... ✅ OK (1 Fundo(s) - Ref: 10/2025, NPL: 18594.29%) 
[3/441] 52362629000185 - 3T FICFIDC NP 1... ✅ OK (1 Fundo(s) - Ref: 11/2025, NPL: 0.00%) 
[4/441] 50356663000111 - ABF VISION FIDC NP SÊNIOR 1... ✅ OK (1 Fundo(s) - Ref: 11/2025, NPL: 1.05%) 
[5/441] 41609997000169 - ACERTO GRUPO INTER FIDC NP SUBORDINADA JÚNIOR 1... ✅ OK (1 Fundo(s) - Ref: 11/2025, NPL: 0.00%) 
[6/441] 24506241000175 - ACL FIDC NP SUBORDINADA JÚNIOR 1... ✅ OK (1 Fundo(s) - Ref: 11/2025, NPL: 0.00%) 
[7/441] 42358004000196 - ACONCÁGUA RESP LIMITADA FIDC NP SÊNIOR 1..

,CNPJ_FUNDO,NOME_FUNDO,STATUS,DATA_REFERENCIA_DOC,ATIVO_TOTAL,INADIMPLENCIA_TOTAL,INDICE_NPL_PERCENTUAL
0,5.119912e+13,2MONEY FUNDO DE INVESTIMENTO EM DIREITOS CREDI...,SUCESSO,11/2025,3.869083e+06,9.567606e+04,0.000000
1,4.738872e+13,3R - FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓ...,SUCESSO,10/2025,6.406223e+06,3.702944e+06,18594.294985
2,5.236263e+13,3T - FUNDO DE INVESTIMENTO EM COTAS DE FUNDO D...,SUCESSO,11/2025,1.297557e+07,0.000000e+00,0.000000
3,5.035666e+13,ABF VISION FUNDO DE INVESTIMENTO EM DIREITOS C...,SUCESSO,11/2025,2.373617e+06,2.021446e+04,1.050868
4,4.161000e+13,ACERTO GRUPO INTER - FUNDO DE INVESTIMENTO EM ...,SUCESSO,11/2025,3.902881e+06,0.000000e+00,0.000000
5,2.450624e+13,ACL NP FUNDO DE INVESTIMENTO EM DIREITOS CREDI...,SUCESSO,11/2025,1.073286e+08,3.501604e+06,0.000000
6,4.235800e+13,ACONCÁGUA FUNDO DE INVESTIMENTO EM DIREITOS CR...,SUCESSO,11/2025,3.466509e+09,3.464305e+09,0.000000
7,3.752923e+13,ACTIUM FUNDO DE INVESTIMENTO EM DIREITOS CREDI...,SUCESSO,11/2025,1.608642e+08,0.000000e+00,0.000000
8,1.342549e+13,ACTIVA FUNDO DE INVESTIMENTO EM DIREITOS CREDI...,SUCESSO,11/2025,2.353727e+07,1.744339e+06,58.151903
9,4.900069e+13,ADVANTAGE CREDIT FUNDO DE INVESTIMENTO EM DIRE...,SUCESSO,11/2025,5.564928e+06,4.519862e+05,9.339769



💾 EXPORTAÇÃO DOS DATOS
────────────────────────────────────────────────────────────────────────────────
   ✅ CSV salvo: base_fidc_completa.csv
      • Encoding: UTF-8 com BOM (compatível com Excel)
      • Separador: ponto-e-vírgula (;)
      • Decimal: vírgula (,)
      • Linhas: 441
      • Colunas: 164
   ✅ CSV (só sucessos): base_fidc_sucesso.csv
      • Linhas: 441
   ✅ Excel salvo: base_fidc_completa.xlsx
      • Abas: Todos (441), Sucesso (441), Erros (0)

════════════════════════════════════════════════════════════════════════════════
🎉 ETL CONCLUÍDO COM SUCESSO!
════════════════════════════════════════════════════════════════════════════════

📥 Para baixar os arquivos:
   1. Clique no ícone de pasta (📁) na barra lateral esquerda
   2. Localize os arquivos CSV/Excel gerados
   3. Clique nos 3 pontinhos (...) ao lado do arquivo
   4. Selecione 'Download'

🔄 Próximos passos:
   1. Importe 'base_fidc_completa.csv' no Power BI / Tableau / Excel
   2. Crie dashboards com os 90+ cam

In [ ]:
import pandas as pd
import requests
from datetime import datetime

# ═══════════════════════════════════════════════════════════════
# SNIPPET: ANÁLISE DE DOCUMENTOS DE DEZEMBRO (ÚLTIMOS 5 ANOS)
# ═══════════════════════════════════════════════════════════════

URL_API_BUSCA = "https://fnet.bmfbovespa.com.br/fnet/publico/pesquisarGerenciadorDocumentosDados"
HEADERS = {"User-Agent": "Mozilla/5.0"}
anos_retroativos = 5 # Define anos_retroativos globally

# Use todos os CNPJs únicos do DataFrame df_cnpjs_entrada
# Se df_cnpjs_entrada não estiver definido (ex: kernel reiniciado), carrega o CSV
if 'df_cnpjs_entrada' not in globals():
    ARQUIVO_CSV_ENTRADA = 'lista_cnpjs_fidc.csv'
    try:
        df_cnpjs_entrada = pd.read_csv(ARQUIVO_CSV_ENTRADA, encoding='utf-8-sig', sep=',')
        df_cnpjs_entrada['CNPJ'] = df_cnpjs_entrada['CNPJ'].astype(str).str.replace('.0', '', regex=False)
        df_cnpjs_entrada['CNPJ'] = df_cnpjs_entrada['CNPJ'].str.zfill(14)
        print(f"[INFO] Arquivo '{ARQUIVO_CSV_ENTRADA}' carregado e CNPJs formatados novamente para análise de dezembro.")
    except FileNotFoundError:
        print(f"❌ ERRO: Arquivo '{ARQUIVO_CSV_ENTRADA}' não encontrado. Por favor, verifique se o arquivo foi carregado.")
        raise
    except Exception as e:
        print(f"❌ ERRO ao carregar ou processar o arquivo '{ARQUIVO_CSV_ENTRADA}': {e}")
        raise

cnpjs_para_analise = df_cnpjs_entrada['CNPJ'].unique().tolist()

def buscar_informes_dezembro(cnpj: str, anos_retroativos: int = 5) -> dict:
    """
    Busca o último informe disponível e todos os informes de dezembro
    dos últimos N anos para um CNPJ.
    """
    try:
        # Buscar documentos na API
        params = {
            'd': 0,
            's': 0,
            'l': 200,
            'cnpjFundo': cnpj
        }

        resp = requests.get(URL_API_BUSCA, params=params, headers=HEADERS, timeout=10)
        resp.raise_for_status()

        data = resp.json().get('data', [])

        if not data:
            return {
                'cnpj': cnpj,
                'status': 'SEM_DADOS',
                'ultimo_mes': None,
                'dezembro_encontrados': []
            }

        df = pd.DataFrame(data)

        # Filtrar apenas 'Informe Mensal Estruturado' e status 'Ativo'
        df_mensal = df[
            (df['tipoDocumento'].str.strip() == "Informe Mensal Estruturado") &
            (df['situacaoDocumento'].str.strip() == "A")
        ].copy()

        if df_mensal.empty:
            return {
                'cnpj': cnpj,
                'status': 'SEM_INFORMES_MENSAIS',
                'ultimo_mes': None,
                'dezembro_encontrados': []
            }

        # Converter datas para ordenação (formato MM/YYYY -> datetime)
        df_mensal['dataReferenciaOrdenavel'] = pd.to_datetime(
            df_mensal['dataReferencia'].str[3:] + '-' +
            df_mensal['dataReferencia'].str[:2] + '-01',
            format='%Y-%m-%d',
            errors='coerce'
        )

        df_mensal = df_mensal.dropna(subset=['dataReferenciaOrdenavel'])
        df_mensal = df_mensal.sort_values('dataReferenciaOrdenavel', ascending=False)

        if df_mensal.empty:
            return {
                'cnpj': cnpj,
                'status': 'SEM_INFORMES_VALIDOS_APOS_DATA_PROCESSAMENTO',
                'ultimo_mes': None,
                'dezembro_encontrados': []
            }

        # Último mês disponível
        ultimo_doc = df_mensal.iloc[0]
        ultimo_mes_info = {
            'data_referencia': ultimo_doc['dataReferencia'],
            'id_documento': ultimo_doc['id'],
            'nome_fundo': ultimo_doc.get('descricaoFundo', 'N/A')
        }

        # Buscar documentos de dezembro dos últimos N anos
        ano_atual = datetime.now().year
        anos_busca = list(range(ano_atual - anos_retroativos, ano_atual))

        dezembro_docs = []
        for ano in anos_busca:
            data_dezembro = f"12/{ano}"
            docs_dezembro = df_mensal[df_mensal['dataReferencia'] == data_dezembro]

            if not docs_dezembro.empty:
                doc = docs_dezembro.iloc[0]
                dezembro_docs.append({
                    'ano': ano,
                    'data_referencia': doc['dataReferencia'],
                    'id_documento': doc['id'],
                    'nome_fundo': doc.get('descricaoFundo', 'N/A'),
                    'encontrado': True
                })
            else:
                dezembro_docs.append({
                    'ano': ano,
                    'data_referencia': data_dezembro,
                    'id_documento': None,
                    'nome_fundo': None,
                    'encontrado': False
                })

        return {
            'cnpj': cnpj,
            'status': 'SUCESSO',
            'ultimo_mes': ultimo_mes_info,
            'dezembro_encontrados': dezembro_docs
        }

    except Exception as e:
        return {
            'cnpj': cnpj,
            'status': 'ERRO',
            'erro_msg': str(e),
            'ultimo_mes': None,
            'dezembro_encontrados': []
        }

# ═══════════════════════════════════════════════════════════════
# PROCESSAMENTO
# ═══════════════════════════════════════════════════════════════

print(f"🔍 BUSCANDO INFORMES DE DEZEMBRO (Últimos {anos_retroativos} anos) para {len(cnpjs_para_analise)} CNPJs\n")
print("═" * 80)

resultados = []

for i, cnpj in enumerate(cnpjs_para_analise, 1):
    print(f"[{i}/{len(cnpjs_para_analise)}] Processando {cnpj}... ", end="")

    resultado = buscar_informes_dezembro(cnpj, anos_retroativos=5)
    resultados.append(resultado)

    if resultado['status'] == 'SUCESSO':
        ultimo = resultado['ultimo_mes']['data_referencia']
        dez_encontrados = sum(1 for d in resultado['dezembro_encontrados'] if d['encontrado'])
        print(f"✅ Último: {ultimo} | Dezembros: {dez_encontrados}/5")
    else:
        print(f"❌ {resultado['status']} ({resultado.get('erro_msg', '')})")

print("═" * 80)

# ═══════════════════════════════════════════════════════════════
# GERAR RELATÓRIO CONSOLIDADO
# ═══════════════════════════════════════════════════════════════

relatorio = []

for res in resultados:
    if res['status'] == 'SUCESSO':
        cnpj = res['cnpj']
        nome_base_ultimo_mes = res['ultimo_mes']['nome_fundo'][:30] # Truncar nome

        # Linha do último mês
        relatorio.append({
            'CNPJ': cnpj,
            'NOME_FUNDO': nome_base_ultimo_mes,
            'TIPO': '📅 ÚLTIMO MÊS',
            'DATA_REF': res['ultimo_mes']['data_referencia'],
            'ID_DOCUMENTO': res['ultimo_mes']['id_documento'],
            'STATUS': '✅ Disponível'
        })

        # Linhas de dezembro
        for dez in res['dezembro_encontrados']:
            relatorio.append({
                'CNPJ': cnpj,
                'NOME_FUNDO': dez['nome_fundo'][:30] if dez['nome_fundo'] else '',
                'TIPO': f'🎄 Dezembro/{dez["ano"]}',
                'DATA_REF': dez['data_referencia'],
                'ID_DOCUMENTO': dez['id_documento'] if dez['encontrado'] else '',
                'STATUS': '✅ Encontrado' if dez['encontrado'] else '❌ Não encontrado'
            })
    else:
        relatorio.append({
            'CNPJ': res['cnpj'],
            'NOME_FUNDO': 'N/A',
            'TIPO': 'ERRO',
            'DATA_REF': 'N/A',
            'ID_DOCUMENTO': 'N/A',
            'STATUS': f"❌ {res['status']} ({res.get('erro_msg', '')})"
        })

df_relatorio = pd.DataFrame(relatorio)

print("\n📊 RELATÓRIO CONSOLIDADO:\n")
print(df_relatorio.to_string(index=False))

# Estatísticas
print("\n📈 ESTATÍSTICAS:")
print(f"   CNPJs processados: {len(cnpjs_para_analise)}")
print(f"   CNPJs com sucesso: {sum(1 for r in resultados if r['status'] == 'SUCESSO')}")

total_dez = sum(len(r['dezembro_encontrados']) for r in resultados if r['status'] == 'SUCESSO')
encontrados_dez = sum(
    sum(1 for d in r['dezembro_encontrados'] if d['encontrado'])
    for r in resultados if r['status'] == 'SUCESSO'
)
print(f"   Dezembros encontrados (sucesso): {encontrados_dez}/{total_dez} ({encontrados_dez/total_dez*100:.1f}%) \n")

# Salvar como XLSX
nome_arquivo_excel = 'analise_dezembro_5anos.xlsx'
try:
    df_relatorio.to_excel(nome_arquivo_excel, index=False)
    print(f"💾 Relatório salvo em: {nome_arquivo_excel}")
except ImportError:
    print(f"\n⚠️  Erro: A biblioteca 'openpyxl' não está instalada. Não foi possível gerar o arquivo Excel.")
    print("   Por favor, instale-a com: !pip install openpyxl")
    # Salvar como CSV como fallback
    nome_arquivo_csv_fallback = 'analise_dezembro_5anos_fallback.csv'
    df_relatorio.to_csv(nome_arquivo_csv_fallback, index=False, encoding='utf-8-sig', sep=';')
    print(f"   Salvo como CSV (fallback): {nome_arquivo_csv_fallback}")

🔍 BUSCANDO INFORMES DE DEZEMBRO (Últimos 5 anos) para 441 CNPJs

════════════════════════════════════════════════════════════════════════════════
[1/441] Processando 51199121000145... ✅ Último: 11/2025 | Dezembros: 2/5
[2/441] Processando 47388724000118... ✅ Último: 10/2025 | Dezembros: 2/5
[3/441] Processando 52362629000185... ✅ Último: 11/2025 | Dezembros: 2/5
[4/441] Processando 50356663000111... ✅ Último: 11/2025 | Dezembros: 1/5
[5/441] Processando 41609997000169... ✅ Último: 11/2025 | Dezembros: 4/5
[6/441] Processando 24506241000175... ✅ Último: 11/2025 | Dezembros: 5/5
[7/441] Processando 42358004000196... ✅ Último: 11/2025 | Dezembros: 3/5
[8/441] Processando 37529228000183... ✅ Último: 11/2025 | Dezembros: 5/5
[9/441] Processando 13425492000109... ✅ Último: 11/2025 | Dezembros: 5/5
[10/441] Processando 49000687000154... ✅ Último: 11/2025 | Dezembros: 2/5
[11/441] Processando 57377862000155... ✅ Último: 11/2025 | Dezembros: 1/5
[12/441] Processando 51774918000129... ✅ Último: 

In [ ]:
# ═════════════════════════════════════════════════════════════════════
# ETL FIDC V4 - EXTRAÇÃO COMPLETA COM COLETA HISTÓRICA
# ═════════════════════════════════════════════════════════════════════
# Autor: Rafael Augusto
# Data: Dezembro 2025
# Objetivo: Extrair dados completos de FIDCs incluindo séries históricas
# Input: lista_cnpjs_fidc.csv (441 CNPJs únicos)
# Output: base_fidc_completa_historica.csv (90+ campos × múltiplos períodos)
# ═════════════════════════════════════════════════════════════════════

# ═════════════════════════════════════════════════════════════════════
# CÉLULA 1: IMPORTS E CONFIGURAÇÕES
# ═════════════════════════════════════════════════════════════════════

import pandas as pd
import requests
import base64
import xml.etree.ElementTree as ET
import time
import os
from typing import Dict, List, Optional
from datetime import datetime

# Configurações da API B3
URL_API_BUSCA = "https://fnet.bmfbovespa.com.br/fnet/publico/pesquisarGerenciadorDocumentosDados"
URL_API_DOWNLOAD = "https://fnet.bmfbovespa.com.br/fnet/publico/downloadDocumento"
HEADERS = {"User-Agent": "Mozilla/5.0"}

# Configurações de controle
DELAY_ENTRE_REQUISICOES = 2  # segundos
TIMEOUT_BUSCA = 10
TIMEOUT_DOWNLOAD = 20

# 🆕 NOVO: Configurações de coleta histórica
ANOS_DEZEMBRO = [2020, 2021, 2022, 2023, 2024]  # Anos de dezembro para coletar
COLETAR_ULTIMO_MES = True  # Se True, coleta também o último mês disponível

print("✅ Bibliotecas importadas com sucesso!")
print(f"📊 Configurações:")
print(f"   • Delay entre requisições: {DELAY_ENTRE_REQUISICOES}s")
print(f"   • Timeout busca: {TIMEOUT_BUSCA}s")
print(f"   • Timeout download: {TIMEOUT_DOWNLOAD}s")
print(f"   🆕 Anos de dezembro: {ANOS_DEZEMBRO}")
print(f"   🆕 Coletar último mês: {'Sim' if COLETAR_ULTIMO_MES else 'Não'}")

# ═════════════════════════════════════════════════════════════════════
# CÉLULA 2: FUNÇÕES AUXILIARES (MANTIDAS IGUAIS)
# ═════════════════════════════════════════════════════════════════════

def limpar_tag(tag: str) -> str:
    """Remove namespace XML das tags."""
    return tag.split('}')[-1] if '}' in tag else tag


def converter_valor(texto: str) -> float:
    """Converte string no formato brasileiro para float."""
    if not texto or not str(texto).strip():
        return 0.0

    try:
        texto_limpo = str(texto).replace('.', '').replace(',', '.')
        return float(texto_limpo)
    except (ValueError, AttributeError):
        return 0.0


def buscar_valor_xml(root: ET.Element, caminho: str) -> any:
    """Busca um valor no XML e tenta converter para float."""
    elemento = root.find(f'.//{caminho}')

    if elemento is not None and elemento.text:
        try:
            return converter_valor(elemento.text)
        except:
            return elemento.text.strip()

    return 0.0 if '/' in caminho else ''


print("✅ Funções auxiliares definidas")

# ═════════════════════════════════════════════════════════════════════
# CÉLULA 3: FUNÇÃO DE EXTRAÇÃO COMPLETA DO XML (MANTIDA IGUAL)
# ═════════════════════════════════════════════════════════════════════

def extrair_dados_xml_completo(xml_content: bytes) -> Dict:
    """Extrai TODOS os campos relevantes do XML de Informe Mensal FIDC."""
    root = ET.fromstring(xml_content)

    dados = {
        # ═══ SEÇÃO 1: IDENTIFICAÇÃO DO FUNDO ═══
        'CNPJ_FUNDO': buscar_valor_xml(root, 'NR_CNPJ_FUNDO'),
        'CNPJ_ADMINISTRADOR': buscar_valor_xml(root, 'NR_CNPJ_ADM'),
        'DATA_COMPETENCIA': buscar_valor_xml(root, 'DT_COMPT'),
        'TIPO_CONDOMINIO': buscar_valor_xml(root, 'TP_CONDOMINIO'),
        'FUNDO_EXCLUSIVO': buscar_valor_xml(root, 'FDO_EXCL'),
        'CLASSE_UNICA': buscar_valor_xml(root, 'CLASS_UNICA'),
        'COTISTA_VINCULADO': buscar_valor_xml(root, 'COTST_VINCUL'),

        # ═══ SEÇÃO 2: ATIVOS GERAIS ═══
        'ATIVO_TOTAL': buscar_valor_xml(root, 'VL_SOM_APLIC_ATIVO'),
        'DISPONIBILIDADES': buscar_valor_xml(root, 'VL_DISPONIB'),
        'CARTEIRA_TOTAL': buscar_valor_xml(root, 'VL_CARTEIRA'),
        'OUTROS_ATIVOS_TOTAL': buscar_valor_xml(root, 'VL_SOM_OUTROS_ATIVOS'),
        'OUTROS_ATIVOS_CURTO_PRAZO': buscar_valor_xml(root, 'OUTROS_ATIVOS/VL_OUTRO_VL_RECEB_CURPRZ'),
        'OUTROS_ATIVOS_LONGO_PRAZO': buscar_valor_xml(root, 'OUTROS_ATIVOS/VL_OUTRO_VL_RECEB_LPRAZO'),

        # ═══ SEÇÃO 3: CRÉDITOS EXISTENTES ═══
        'CREDITOS_ADQUIRIDOS': buscar_valor_xml(root, 'CRED_EXISTE/VL_SOM_DICRED_AQUIS'),
        'CRED_VENCIDOS_ADIMPLENTES': buscar_valor_xml(root, 'CRED_EXISTE/VL_CRED_EXISTE_VENC_ADIMPL'),
        'CRED_VENCIDOS_INADIMPLENTES': buscar_valor_xml(root, 'CRED_EXISTE/VL_CRED_EXISTE_VENC_INAD'),
        'CRED_TOTAL_VENC_INADIMPL': buscar_valor_xml(root, 'CRED_EXISTE/VL_CRED_TOTAL_VENC_INAD'),
        'CRED_INADIMPLENCIA': buscar_valor_xml(root, 'CRED_EXISTE/VL_CRED_EXISTE_INAD'),
        'CRED_PERFORMADOS': buscar_valor_xml(root, 'CRED_EXISTE/VL_CRED_REFER_DICRED_PERFO'),
        'CRED_VENCIDOS_PENDENTES': buscar_valor_xml(root, 'CRED_EXISTE/VL_CRED_VENC_PEND'),
        'CRED_EMP_RECUPERACAO': buscar_valor_xml(root, 'CRED_EXISTE/VL_CRED_ORIGEM_EMP_PROC_RECUP'),
        'CRED_RECEITA_PUBLICA': buscar_valor_xml(root, 'CRED_EXISTE/VL_DECOR_RECEIT_PUBLIC'),
        'CRED_ACAO_JUDICIAL': buscar_valor_xml(root, 'CRED_EXISTE/VL_CRED_ACAO_JUDIC'),
        'CRED_CONSTITUICAO_JURIDICA': buscar_valor_xml(root, 'CRED_EXISTE/VL_CRED_CONST_JUR_FATRISC'),
        'CRED_PROVISAO_REDUCAO': buscar_valor_xml(root, 'CRED_EXISTE/VL_PROVIS_REDUC_RECUP'),

        # ═══ SEÇÃO 4: DIREITOS CREDITÓRIOS (DICRED) ═══
        'DICRED_TOTAL': buscar_valor_xml(root, 'DICRED/VL_DICRED'),
        'DICRED_CEDENTE': buscar_valor_xml(root, 'DICRED/VL_DICRED_CEDENT'),
        'DICRED_VENC_INADIMPL': buscar_valor_xml(root, 'DICRED/VL_DICRED_EXISTE_VENC_INAD'),
        'DICRED_TOTAL_VENC_INAD': buscar_valor_xml(root, 'DICRED/VL_DICRED_TOTAL_VENC_INAD'),
        'DICRED_INADIMPLENCIA': buscar_valor_xml(root, 'DICRED/VL_DICRED_EXISTE_INAD'),
        'DICRED_PERFORMADOS': buscar_valor_xml(root, 'DICRED/VL_DICRED_REFER_DICRED_PERFO'),
        'DICRED_VENC_PENDENTES': buscar_valor_xml(root, 'DICRED/VL_DICRED_VENC_PEND'),
        'DICRED_EMP_RECUPERACAO': buscar_valor_xml(root, 'DICRED/VL_DICRED_ORIGEM_EMP_PROC_RECUP'),
        'DICRED_RECEITA_PUBLICA': buscar_valor_xml(root, 'DICRED/VL_DICRED_RECEIT_PUBLIC'),
        'DICRED_ACAO_JUDICIAL': buscar_valor_xml(root, 'DICRED/VL_DICRED_ACAO_JUDIC'),
        'DICRED_PROVISAO_REDUCAO': buscar_valor_xml(root, 'DICRED/VL_DICRED_PROVIS_REDUC_RECUP'),

        # ═══ SEÇÃO 5: VALORES MOBILIÁRIOS ═══
        'VALORES_MOBILIARIOS_TOTAL': buscar_valor_xml(root, 'VALORES_MOB/VL_SOM_VALORES_MOB'),
        'DEBENTURES': buscar_valor_xml(root, 'VALORES_MOB/VL_DEBT'),
        'CRI': buscar_valor_xml(root, 'VALORES_MOB/VL_CRI'),
        'NOTAS_PROMISSORIAS_COMERCIAIS': buscar_valor_xml(root, 'VALORES_MOB/VL_NP_COMERC'),
        'LETRAS_FINANCEIRAS': buscar_valor_xml(root, 'VALORES_MOB/VL_LETRA_FINANC'),
        'COTAS_FIF': buscar_valor_xml(root, 'VALORES_MOB/VL_CLS_COTA_FIF'),
        'OUTROS_DIREITOS_CREDITORIOS': buscar_valor_xml(root, 'VALORES_MOB/VL_OUTRO_DICRED'),

        # ═══ SEÇÃO 6: OUTROS ATIVOS FINANCEIROS ═══
        'TITULOS_PUBLICOS_FEDERAIS': buscar_valor_xml(root, 'VL_TITPUB_FED'),
        'CDB': buscar_valor_xml(root, 'VL_CDB'),
        'APLICACOES_COMPROMISSADAS': buscar_valor_xml(root, 'VL_APLIC_OPER_COMPSS'),
        'ATIVOS_FINANCEIROS_RF': buscar_valor_xml(root, 'VL_ATIV_FINANC_RF'),
        'COTAS_FIDC': buscar_valor_xml(root, 'VL_COTA_FIDC'),

        # ═══ SEÇÃO 7: MERCADO DE DERIVATIVOS ═══
        'DERIVATIVOS_TOTAL': buscar_valor_xml(root, 'MERC_DERIVATIVO/VL_SOM_MERC_DERIVATIVO'),
        'TERMO_COMPRADOR': buscar_valor_xml(root, 'MERC_DERIVATIVO/VL_MERC_TERMO_POS_COMPRD'),
        'OPCOES_TITULAR': buscar_valor_xml(root, 'MERC_DERIVATIVO/VL_MERC_OP_POS_TITUL'),
        'FUTUROS_AJUSTE_POSITIVO': buscar_valor_xml(root, 'MERC_DERIVATIVO/VL_MERC_FUT_AJUST_POSIT'),
        'SWAP_A_RECEBER': buscar_valor_xml(root, 'MERC_DERIVATIVO/VL_DIFER_SWAP_RECEB'),
        'COBERTURA_PRESTADA': buscar_valor_xml(root, 'MERC_DERIVATIVO/VL_COBERT_PREST'),
        'DEPOSITOS_MARGEM': buscar_valor_xml(root, 'MERC_DERIVATIVO/VL_DEPOS_MARGEM'),

        # ═══ SEÇÃO 8: SEGMENTAÇÃO DA CARTEIRA ═══
        'CARTEIRA_SEGMENTADA_TOTAL': buscar_valor_xml(root, 'CART_SEGMT/VL_SOM_CART_SEGMT'),
        'SEGMT_INDUSTRIAL': buscar_valor_xml(root, 'CART_SEGMT/VL_IND'),
        'SEGMT_MERCADO_IMOBILIARIO': buscar_valor_xml(root, 'CART_SEGMT/VL_MERC_IMOBIL'),
        'SEGMT_AGRONEGOCIO': buscar_valor_xml(root, 'CART_SEGMT/VL_AGRONEG'),
        'SEGMT_CARTAO_CREDITO': buscar_valor_xml(root, 'CART_SEGMT/VL_CART_CRED'),
        'SEGMT_ACAO_JUDICIAL': buscar_valor_xml(root, 'CART_SEGMT/VL_ACAO_JUDIC'),
        'SEGMT_PROPRIEDADE_INTELECTUAL': buscar_valor_xml(root, 'CART_SEGMT/VL_PROPRD_MARCA_PATENT'),

        # Subsegmento: COMERCIAL
        'SEGMT_COMERCIAL_TOTAL': buscar_valor_xml(root, 'SEGMT_COMERC/VL_SOM_SEGMT_COMERC'),
        'SEGMT_COMERCIO': buscar_valor_xml(root, 'SEGMT_COMERC/VL_COMERC'),
        'SEGMT_COMERCIO_VAREJO': buscar_valor_xml(root, 'SEGMT_COMERC/VL_COMERC_VARJ'),
        'SEGMT_ARREND_MERCANTIL': buscar_valor_xml(root, 'SEGMT_COMERC/VL_ARREND_MERCNT'),

        # Subsegmento: SERVIÇOS
        'SEGMT_SERVICOS_TOTAL': buscar_valor_xml(root, 'SEGMT_SERV/VL_SOM_SEGMT_SERV'),
        'SEGMT_SERVICOS_GERAIS': buscar_valor_xml(root, 'SEGMT_SERV/VL_SERV'),
        'SEGMT_SERVICOS_PUBLICOS': buscar_valor_xml(root, 'SEGMT_SERV/VL_SERV_PUBLIC'),
        'SEGMT_SERVICOS_EDUCACAO': buscar_valor_xml(root, 'SEGMT_SERV/VL_SERV_EDUC'),
        'SEGMT_SERVICOS_ENTRETENIMENTO': buscar_valor_xml(root, 'SEGMT_SERV/VL_SERV_ENTRETEN'),

        # Subsegmento: FINANCEIRO
        'SEGMT_FINANCEIRO_TOTAL': buscar_valor_xml(root, 'SEGMT_FINANC/VL_SOM_SEGMT_FINANC'),
        'SEGMT_FINANC_CREDITO_PESSOA': buscar_valor_xml(root, 'SEGMT_FINANC/VL_FINANC_CRED_PESSOA'),
        'SEGMT_FINANC_CONSIGNADO': buscar_valor_xml(root, 'SEGMT_FINANC/VL_FINANC_CRED_PESSOA_CONSIG'),
        'SEGMT_FINANC_CORPORATIVO': buscar_valor_xml(root, 'SEGMT_FINANC/VL_FINANC_CRED_CORPOR'),
        'SEGMT_FINANC_MIDDLE_MARKET': buscar_valor_xml(root, 'SEGMT_FINANC/VL_FINANC_MMARKET'),
        'SEGMT_FINANC_VEICULOS': buscar_valor_xml(root, 'SEGMT_FINANC/VL_FINANC_VEICL'),
        'SEGMT_FINANC_IMOB_EMPRESARIAL': buscar_valor_xml(root, 'SEGMT_FINANC/VL_FINANC_IMOBIL_EMPSRL'),
        'SEGMT_FINANC_IMOB_RESIDENCIAL': buscar_valor_xml(root, 'SEGMT_FINANC/VL_FINANC_IMOBIL_RESID'),
        'SEGMT_FINANC_OUTROS': buscar_valor_xml(root, 'SEGMT_FINANC/VL_FINANC_OUTRO'),

        # Subsegmento: FACTORING
        'SEGMT_FACTORING_TOTAL': buscar_valor_xml(root, 'SEGMT_FACT/VL_SOM_SEGMT_FACT'),
        'SEGMT_FACTORING_PESSOA': buscar_valor_xml(root, 'SEGMT_FACT/VL_FACT_PESSOA'),
        'SEGMT_FACTORING_CORPORATIVO': buscar_valor_xml(root, 'SEGMT_FACT/VL_FACT_CORPOR'),

        # Subsegmento: SETOR PÚBLICO
        'SEGMT_SETOR_PUBLICO_TOTAL': buscar_valor_xml(root, 'SEGMT_SETOR_PUBLIC/VL_SOM_SEGMT_SETOR_PUBLIC'),
        'SEGMT_PRECATORIOS': buscar_valor_xml(root, 'SEGMT_SETOR_PUBLIC/VL_SETOR_PUBLIC_PRECAT'),
        'SEGMT_CREDITOS_TRIBUTARIOS': buscar_valor_xml(root, 'SEGMT_SETOR_PUBLIC/VL_SETOR_PUBLIC_CRED_TRIBUT'),
        'SEGMT_ROYALTIES': buscar_valor_xml(root, 'SEGMT_SETOR_PUBLIC/VL_SETOR_PUBLIC_ROYA'),
        'SEGMT_SETOR_PUBLICO_OUTROS': buscar_valor_xml(root, 'SEGMT_SETOR_PUBLIC/VL_SETOR_PUBLIC_OUTRO'),
    }

    # CÁLCULO DE INDICADORES
    inadimpl_cred = dados['CRED_INADIMPLENCIA']
    inadimpl_dicred = dados['DICRED_INADIMPLENCIA']
    dados['INADIMPLENCIA_TOTAL'] = max(inadimpl_cred, inadimpl_dicred)

    carteira_credito = dados['CREDITOS_ADQUIRIDOS']

    # Índice de NPL
    if carteira_credito > 0 and dados['INADIMPLENCIA_TOTAL'] > 0:
        dados['INDICE_NPL_PERCENTUAL'] = (dados['INADIMPLENCIA_TOTAL'] / carteira_credito) * 100
    else:
        dados['INDICE_NPL_PERCENTUAL'] = 0.0

    # Taxa de liquidez
    if dados['ATIVO_TOTAL'] > 0:
        dados['TAXA_LIQUIDEZ_PERCENTUAL'] = (dados['DISPONIBILIDADES'] / dados['ATIVO_TOTAL']) * 100
    else:
        dados['TAXA_LIQUIDEZ_PERCENTUAL'] = 0.0

    # Concentração em crédito
    if dados['ATIVO_TOTAL'] > 0:
        dados['CONCENTRACAO_CREDITO_PERCENTUAL'] = (carteira_credito / dados['ATIVO_TOTAL']) * 100
    else:
        dados['CONCENTRACAO_CREDITO_PERCENTUAL'] = 0.0

    return dados


print("✅ Função de extração completa definida (90+ campos)")

# ═════════════════════════════════════════════════════════════════════
# CÉLULA 4: FUNÇÃO ETL COM COLETA HISTÓRICA (🆕 MODIFICADA)
# ═════════════════════════════════════════════════════════════════════

def etl_fidc_historico(cnpj_alvo: str, nome_fundo_referencia: str = "",
                       anos_dezembro: list = None, coletar_ultimo: bool = True) -> List[Dict]:
    """
    Executa ETL completo para um CNPJ, coletando:
    - Último informe mensal disponível (opcional)
    - Informes de dezembro dos anos especificados

    Args:
        cnpj_alvo: CNPJ do fundo (string com 14 dígitos)
        nome_fundo_referencia: Nome do fundo (opcional)
        anos_dezembro: Lista de anos para buscar dezembro
        coletar_ultimo: Se True, coleta também o último mês disponível

    Returns:
        Lista de dicionários com dados extraídos (um por documento)
    """

    resultados_fundo_cnpj = []

    # Definir anos padrão se não especificado
    if anos_dezembro is None:
        anos_dezembro = ANOS_DEZEMBRO

    try:
        # ─────────────────────────────────────────────────────────────────────
        # ETAPA 1: DISCOVERY
        # ─────────────────────────────────────────────────────────────────────

        params = {
            'd': 0,
            's': 0,
            'l': 200,
            'cnpjFundo': cnpj_alvo
        }

        resp_busca = requests.get(
            URL_API_BUSCA,
            params=params,
            headers=HEADERS,
            timeout=TIMEOUT_BUSCA
        )
        resp_busca.raise_for_status()

        data = resp_busca.json().get('data', [])

        if not data:
            return []

        df_docs = pd.DataFrame(data)

        # ─────────────────────────────────────────────────────────────────────
        # ETAPA 2: FILTER
        # ─────────────────────────────────────────────────────────────────────

        df_mensal = df_docs[
            (df_docs['tipoDocumento'].str.strip() == "Informe Mensal Estruturado") &
            (df_docs['situacaoDocumento'].str.strip() == "A")
        ].copy()

        if df_mensal.empty:
            return []

        # Converter datas
        df_mensal['dataReferenciaOrdenavel'] = pd.to_datetime(
            df_mensal['dataReferencia'].str[3:] + '-' +
            df_mensal['dataReferencia'].str[:2] + '-01',
            format='%Y-%m-%d', errors='coerce'
        )

        df_mensal.dropna(subset=['dataReferenciaOrdenavel'], inplace=True)

        if df_mensal.empty:
            return []

        # ─────────────────────────────────────────────────────────────────────
        # ETAPA 3: SELEÇÃO DE DOCUMENTOS (🆕 LÓGICA MODIFICADA)
        # ─────────────────────────────────────────────────────────────────────

        documentos_processar = []

        # 3.1 - Último mês disponível
        if coletar_ultimo:
            df_mensal_sorted = df_mensal.sort_values('dataReferenciaOrdenavel', ascending=False)
            df_ultimo = df_mensal_sorted.groupby('descricaoFundo').first().reset_index()
            documentos_processar.extend(df_ultimo.to_dict('records'))

        # 3.2 - Documentos de dezembro dos anos especificados
        for ano in anos_dezembro:
            data_dezembro = f"12/{ano}"
            docs_dezembro = df_mensal[df_mensal['dataReferencia'] == data_dezembro]

            if not docs_dezembro.empty:
                df_dez_unico = docs_dezembro.groupby('descricaoFundo').first().reset_index()
                documentos_processar.extend(df_dez_unico.to_dict('records'))

        # Remover duplicatas por ID
        ids_processados = set()
        docs_unicos = []
        for doc in documentos_processar:
            doc_id = doc['id']
            if doc_id not in ids_processados:
                docs_unicos.append(doc)
                ids_processados.add(doc_id)

        # ─────────────────────────────────────────────────────────────────────
        # ETAPA 4-6: DOWNLOAD, EXTRACT, CLEANUP
        # ─────────────────────────────────────────────────────────────────────

        for doc_info in docs_unicos:
            doc_id = doc_info['id']
            data_referencia = doc_info.get('dataReferencia', 'N/A')
            nome_fundo_doc = doc_info.get('descricaoFundo', 'Nome N/A')

            nome_arquivo_temp = None

            try:
                # Download
                url_download = f"{URL_API_DOWNLOAD}?id={doc_id}"
                resp_download = requests.get(
                    url_download,
                    headers=HEADERS,
                    timeout=TIMEOUT_DOWNLOAD
                )
                resp_download.raise_for_status()

                xml_content = base64.b64decode(resp_download.content)

                # Salvar temporariamente
                nome_arquivo_temp = f'temp_{cnpj_alvo}_{doc_id}.xml'
                with open(nome_arquivo_temp, 'wb') as f:
                    f.write(xml_content)

                # Extrair dados
                dados_extraidos = extrair_dados_xml_completo(xml_content)

                if not dados_extraidos.get('CNPJ_FUNDO'):
                    dados_extraidos['CNPJ_FUNDO'] = cnpj_alvo

                # Metadados
                dados_extraidos['NOME_FUNDO'] = nome_fundo_doc
                dados_extraidos['STATUS'] = 'SUCESSO'
                dados_extraidos['ID_DOCUMENTO'] = doc_id
                dados_extraidos['DATA_REFERENCIA_DOC'] = data_referencia
                dados_extraidos['MENSAGEM_ERRO'] = None

                # 🆕 NOVO: Identificar tipo de coleta
                if data_referencia.startswith('12/'):
                    ano = data_referencia.split('/')[1]
                    dados_extraidos['TIPO_COLETA'] = f'DEZEMBRO_{ano}'
                else:
                    dados_extraidos['TIPO_COLETA'] = 'ULTIMO_MES'

                resultados_fundo_cnpj.append(dados_extraidos)

            except requests.exceptions.HTTPError as e:
                resultados_fundo_cnpj.append({
                    'CNPJ_FUNDO': cnpj_alvo,
                    'NOME_FUNDO': nome_fundo_doc,
                    'STATUS': 'ERRO_HTTP_DOWNLOAD',
                    'MENSAGEM_ERRO': f'HTTP Error: {str(e)}',
                    'DATA_REFERENCIA_DOC': data_referencia,
                    'TIPO_COLETA': 'ERRO'
                })
            except requests.exceptions.Timeout as e:
                resultados_fundo_cnpj.append({
                    'CNPJ_FUNDO': cnpj_alvo,
                    'NOME_FUNDO': nome_fundo_doc,
                    'STATUS': 'TIMEOUT_DOWNLOAD',
                    'MENSAGEM_ERRO': f'Timeout: {str(e)}',
                    'DATA_REFERENCIA_DOC': data_referencia,
                    'TIPO_COLETA': 'ERRO'
                })
            except ET.ParseError as e:
                resultados_fundo_cnpj.append({
                    'CNPJ_FUNDO': cnpj_alvo,
                    'NOME_FUNDO': nome_fundo_doc,
                    'STATUS': 'ERRO_PARSE_XML',
                    'MENSAGEM_ERRO': f'Erro XML: {str(e)}',
                    'DATA_REFERENCIA_DOC': data_referencia,
                    'TIPO_COLETA': 'ERRO'
                })
            except Exception as e:
                resultados_fundo_cnpj.append({
                    'CNPJ_FUNDO': cnpj_alvo,
                    'NOME_FUNDO': nome_fundo_doc,
                    'STATUS': 'ERRO_INESPERADO',
                    'MENSAGEM_ERRO': f'Erro: {str(e)}',
                    'DATA_REFERENCIA_DOC': data_referencia,
                    'TIPO_COLETA': 'ERRO'
                })

            finally:
                # Cleanup
                if nome_arquivo_temp and os.path.exists(nome_arquivo_temp):
                    try:
                        os.remove(nome_arquivo_temp)
                    except OSError:
                        pass

    except Exception as e:
        resultados_fundo_cnpj.append({
            'CNPJ_FUNDO': cnpj_alvo,
            'NOME_FUNDO': nome_fundo_referencia,
            'STATUS': 'ERRO_GERAL',
            'MENSAGEM_ERRO': f'Erro geral: {str(e)}',
            'TIPO_COLETA': 'ERRO'
        })

    return resultados_fundo_cnpj


print("✅ Função ETL com coleta histórica definida")

# ═════════════════════════════════════════════════════════════════════
# CÉLULA 5: CARREGAMENTO DOS CNPJs (MANTIDA IGUAL)
# ═════════════════════════════════════════════════════════════════════

ARQUIVO_CSV_ENTRADA = 'lista_cnpjs_fidc.csv'

try:
    df_cnpjs_entrada = pd.read_csv(ARQUIVO_CSV_ENTRADA, encoding='utf-8-sig')

    print(f"✅ Arquivo '{ARQUIVO_CSV_ENTRADA}' carregado com sucesso!")
    print(f"   • Total de CNPJs: {len(df_cnpjs_entrada)}")

    df_cnpjs_entrada['CNPJ'] = df_cnpjs_entrada['CNPJ'].astype(str).str.replace('.0', '', regex=False)
    df_cnpjs_entrada['CNPJ'] = df_cnpjs_entrada['CNPJ'].str.zfill(14)

    print(f"\n📋 Primeiras 5 linhas:")
    print(df_cnpjs_entrada.head())

    # 🆕 Cálculo de tempo ajustado
    docs_estimados_por_cnpj = (1 if COLETAR_ULTIMO_MES else 0) + len(ANOS_DEZEMBRO)
    tempo_estimado = len(df_cnpjs_entrada) * DELAY_ENTRE_REQUISICOES * docs_estimados_por_cnpj / 60

    print(f"\n⏱️  Tempo estimado de processamento:")
    print(f"   • {tempo_estimado:.1f} minutos")
    print(f"   • ~{docs_estimados_por_cnpj} documentos por CNPJ")

except FileNotFoundError:
    print(f"❌ ERRO: Arquivo '{ARQUIVO_CSV_ENTRADA}' não encontrado!")
    raise

# ═════════════════════════════════════════════════════════════════════
# CÉLULA 6: PROCESSAMENTO EM LOTE (🆕 MODIFICADA)
# ═════════════════════════════════════════════════════════════════════

print(f"\n{'═'*80}")
print(f"🚀 INICIANDO PROCESSAMENTO COM COLETA HISTÓRICA")
print(f"   • Total de CNPJs: {len(df_cnpjs_entrada)}")
print(f"   • Anos de dezembro: {ANOS_DEZEMBRO}")
print(f"   • Coletar último mês: {'Sim' if COLETAR_ULTIMO_MES else 'Não'}")
print(f"{'═'*80}\n")

resultados_consolidados = []

for i, row in df_cnpjs_entrada.iterrows():
    cnpj = row['CNPJ']
    nome_fundo_ref = row.get('NOME_FUNDO', '')

    nome_display = nome_fundo_ref[:47] + "..." if len(nome_fundo_ref) > 50 else nome_fundo_ref

    print(f"[{i+1}/{len(df_cnpjs_entrada)}] {cnpj} - {nome_display}... ", end="")

    # 🆕 Usar função modificada
    resultados_atuais = etl_fidc_historico(
        cnpj,
        nome_fundo_ref,
        anos_dezembro=ANOS_DEZEMBRO,
        coletar_ultimo=COLETAR_ULTIMO_MES
    )

    if resultados_atuais:
        resultados_consolidados.extend(resultados_atuais)

        # Contar sucessos
        sucessos = [r for r in resultados_atuais if r.get('STATUS') == 'SUCESSO']

        if sucessos:
            num_docs = len(sucessos)
            # Separar por tipo
            ultimos = sum(1 for s in sucessos if s.get('TIPO_COLETA') == 'ULTIMO_MES')
            dezembros = num_docs - ultimos
            print(f"✅ OK ({num_docs} docs: {ultimos} último + {dezembros} dez)")
        else:
            primeiro_erro = resultados_atuais[0].get('STATUS', 'ERRO')
            print(f"❌ {primeiro_erro}")
    else:
        resultados_consolidados.append({
            'CNPJ_FUNDO': cnpj,
            'NOME_FUNDO': nome_fundo_ref,
            'STATUS': 'NENHUM_DOCUMENTO',
            'TIPO_COLETA': 'ERRO'
        })
        print(f"❌ SEM_DOCS")

    # Delay
    if i < len(df_cnpjs_entrada) - 1:
        time.sleep(DELAY_ENTRE_REQUISICOES)

    # Checkpoint
    if (i + 1) % 50 == 0:
        df_temp = pd.DataFrame(resultados_consolidados)
        df_temp.to_csv('checkpoint_historico.csv', index=False, encoding='utf-8-sig', sep=';', decimal=',')
        print(f"   💾 Checkpoint salvo ({i+1} CNPJs processados)")

print(f"\n{'═'*80}")
print(f"✅ PROCESSAMENTO CONCLUÍDO!")
print(f"{'═'*80}\n")

# ═════════════════════════════════════════════════════════════════════
# CÉLULA 7: CONSOLIDAÇÃO E ESTATÍSTICAS (🆕 MODIFICADA)
# ═════════════════════════════════════════════════════════════════════

df_resultado_final = pd.DataFrame(resultados_consolidados)

# Estatísticas de processamento
total_documentos = len(df_resultado_final)
total_sucesso = len(df_resultado_final[df_resultado_final['STATUS'] == 'SUCESSO'])
total_erros = total_documentos - total_sucesso

print(f"📊 ESTATÍSTICAS DO PROCESSAMENTO")
print(f"{'─'*80}")
print(f"   Total de CNPJs processados: {len(df_cnpjs_entrada)}")
print(f"   Total de documentos coletados: {total_documentos}")
print(f"   ✅ Sucesso: {total_sucesso} ({total_sucesso/total_documentos*100:.1f}%)")
print(f"   ❌ Erros: {total_erros} ({total_erros/total_documentos*100:.1f}%)")

# 🆕 Estatísticas por tipo de coleta
if total_sucesso > 0:
    df_sucesso = df_resultado_final[df_resultado_final['STATUS'] == 'SUCESSO'].copy()

    print(f"\n📅 DISTRIBUIÇÃO POR TIPO DE COLETA:")
    tipos_coleta = df_sucesso['TIPO_COLETA'].value_counts()
    for tipo, count in tipos_coleta.items():
        print(f"   • {tipo}: {count} documentos")

# Distribuição de erros
if total_erros > 0:
    print(f"\n⚠️  TIPOS DE ERROS:")
    erros_por_tipo = df_resultado_final[df_resultado_final['STATUS'] != 'SUCESSO']['STATUS'].value_counts()
    for status, count in erros_por_tipo.items():
        print(f"   • {status}: {count}")

# Estatísticas financeiras
if total_sucesso > 0:
    print(f"\n💰 ESTATÍSTICAS FINANCEIRAS")
    print(f"{'─'*80}")

    ativo_total_soma = df_sucesso['ATIVO_TOTAL'].sum()
    ativo_total_media = df_sucesso['ATIVO_TOTAL'].mean()

    inadimpl_soma = df_sucesso['INADIMPLENCIA_TOTAL'].sum()
    npl_medio = df_sucesso['INDICE_NPL_PERCENTUAL'].mean()

    print(f"   Ativo Total (Soma):     R$ {ativo_total_soma:,.2f}")
    print(f"   Ativo Total (Média):    R$ {ativo_total_media:,.2f}")
    print(f"   Inadimplência (Soma):   R$ {inadimpl_soma:,.2f}")
    print(f"   NPL Médio:              {npl_medio:.2f}%")

# ═════════════════════════════════════════════════════════════════════
# CÉLULA 8: EXPORTAÇÃO (🆕 MODIFICADA)
# ═════════════════════════════════════════════════════════════════════

# CSV principal
arquivo_csv = 'base_fidc_completa_historica.csv'
df_resultado_final.to_csv(arquivo_csv, index=False, encoding='utf-8-sig', sep=';', decimal=',')
print(f"\n💾 Arquivo salvo: {arquivo_csv}")

# Excel com múltiplas abas
arquivo_excel = 'base_fidc_completa_historica.xlsx'

with pd.ExcelWriter(arquivo_excel, engine='openpyxl') as writer:
    # Aba 1: Todos os dados
    df_resultado_final.to_excel(writer, sheet_name='Dados_Completos', index=False)

    # Aba 2: Apenas sucessos
    if total_sucesso > 0:
        df_sucesso.to_excel(writer, sheet_name='Dados_Validos', index=False)

    # Aba 3: Apenas erros
    if total_erros > 0:
        df_erros = df_resultado_final[df_resultado_final['STATUS'] != 'SUCESSO']
        df_erros.to_excel(writer, sheet_name='Erros', index=False)

    # 🆕 Aba 4: Últimos meses
    if COLETAR_ULTIMO_MES and total_sucesso > 0:
        df_ultimo = df_sucesso[df_sucesso['TIPO_COLETA'] == 'ULTIMO_MES']
        if not df_ultimo.empty:
            df_ultimo.to_excel(writer, sheet_name='Ultimo_Mes', index=False)

    # 🆕 Aba 5-9: Dezembros por ano
    for ano in ANOS_DEZEMBRO:
        tipo_coleta = f'DEZEMBRO_{ano}'
        df_ano = df_sucesso[df_sucesso['TIPO_COLETA'] == tipo_coleta]
        if not df_ano.empty:
            df_ano.to_excel(writer, sheet_name=f'Dezembro_{ano}', index=False)

print(f"💾 Arquivo salvo: {arquivo_excel}")
print(f"\n✅ PROCESSAMENTO CONCLUÍDO COM SUCESSO!")


✅ Bibliotecas importadas com sucesso!
📊 Configurações:
   • Delay entre requisições: 2s
   • Timeout busca: 10s
   • Timeout download: 20s
   🆕 Anos de dezembro: [2020, 2021, 2022, 2023, 2024]
   🆕 Coletar último mês: Sim
✅ Funções auxiliares definidas
✅ Função de extração completa definida (90+ campos)
✅ Função ETL com coleta histórica definida
✅ Arquivo 'lista_cnpjs_fidc.csv' carregado com sucesso!
   • Total de CNPJs: 441

📋 Primeiras 5 linhas:
             CNPJ                                         NOME_FUNDO  \
0  51199121000145  2MONEY RESP LIMITADA FIDC NP SUBORDINADA JÚNIOR 1   
1  47388724000118                   3R RESP LIMITADA FIDC NP ÚNICA 1   
2  52362629000185                                    3T FICFIDC NP 1   
3  50356663000111                        ABF VISION FIDC NP SÊNIOR 1   
4  41609997000169    ACERTO GRUPO INTER FIDC NP SUBORDINADA JÚNIOR 1   

        CNPJ_ORIGINAL  
0  51.199.121/0001-45  
1  47.388.724/0001-18  
2  52.362.629/0001-85  
3  50.356.663/0001-

In [ ]:
# ═════════════════════════════════════════════════════════════════════
# ETL FIDC V5 - VERSÃO OTIMIZADA PARA GOOGLE COLAB
# ═════════════════════════════════════════════════════════════════════
# Melhorias implementadas:
# 1. ✅ Bug fix - Conversão de tipos
# 2. ✅ Logging profissional
# 3. ✅ Retry logic com backoff exponencial
# 4. ✅ Paralelização (ThreadPoolExecutor)
# 5. ✅ Cache de resultados
# 6. ✅ Progress bar (tqdm)
# 7. ✅ Validação robusta de dados
# ═════════════════════════════════════════════════════════════════════

# ═════════════════════════════════════════════════════════════════════
# CÉLULA 1: INSTALAÇÃO DE DEPENDÊNCIAS (APENAS SE NECESSÁRIO)
# ═════════════════════════════════════════════════════════════════════

# Instalar apenas se não tiver (Colab já tem a maioria)
!pip install tenacity -q

print("✅ Dependências verificadas!")

# ═════════════════════════════════════════════════════════════════════
# CÉLULA 2: IMPORTS E CONFIGURAÇÕES
# ═════════════════════════════════════════════════════════════════════

import pandas as pd
import requests
import base64
import xml.etree.ElementTree as ET
import time
import os
import logging
import hashlib
import pickle
from typing import Dict, List, Optional, Union
from datetime import datetime
from pathlib import Path
from concurrent.futures import ThreadPoolExecutor, as_completed
from tenacity import retry, stop_after_attempt, wait_exponential, retry_if_exception_type
from tqdm.auto import tqdm
import warnings
warnings.filterwarnings('ignore')

# ═════════════════════════════════════════════════════════════════════
# CONFIGURAÇÕES
# ═════════════════════════════════════════════════════════════════════

# API B3
URL_API_BUSCA = "https://fnet.bmfbovespa.com.br/fnet/publico/pesquisarGerenciadorDocumentosDados"
URL_API_DOWNLOAD = "https://fnet.bmfbovespa.com.br/fnet/publico/downloadDocumento"
HEADERS = {"User-Agent": "Mozilla/5.0"}

# Controle de requisições
DELAY_ENTRE_REQUISICOES = 2
TIMEOUT_BUSCA = 10
TIMEOUT_DOWNLOAD = 20
MAX_RETRIES = 3

# Coleta histórica
ANOS_DEZEMBRO = [2020, 2021, 2022, 2023, 2024]
COLETAR_ULTIMO_MES = True

# Performance
MAX_WORKERS = 5  # Threads paralelas
CACHE_ENABLED = True

# ═════════════════════════════════════════════════════════════════════
# CONFIGURAÇÃO DE LOGGING
# ═════════════════════════════════════════════════════════════════════

# Criar diretório de logs
LOG_DIR = Path('logs')
LOG_DIR.mkdir(exist_ok=True)

# Configurar logger
log_filename = LOG_DIR / f'etl_fidc_{datetime.now():%Y%m%d_%H%M%S}.log'

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s | %(levelname)-8s | %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S',
    handlers=[
        logging.FileHandler(log_filename, encoding='utf-8'),
        logging.StreamHandler()
    ]
)

logger = logging.getLogger(__name__)

# Configurar cache
CACHE_DIR = Path('.cache_fidc')
if CACHE_ENABLED:
    CACHE_DIR.mkdir(exist_ok=True)
    logger.info(f"Cache habilitado em: {CACHE_DIR.absolute()}")

logger.info("="*80)
logger.info("ETL FIDC V5 - VERSÃO OTIMIZADA")
logger.info("="*80)
logger.info(f"Configurações carregadas:")
logger.info(f"  • Delay entre requisições: {DELAY_ENTRE_REQUISICOES}s")
logger.info(f"  • Max workers (threads): {MAX_WORKERS}")
logger.info(f"  • Max retries: {MAX_RETRIES}")
logger.info(f"  • Anos dezembro: {ANOS_DEZEMBRO}")
logger.info(f"  • Cache: {'Habilitado' if CACHE_ENABLED else 'Desabilitado'}")

# ═════════════════════════════════════════════════════════════════════
# CÉLULA 3: FUNÇÕES AUXILIARES MELHORADAS
# ═════════════════════════════════════════════════════════════════════

def limpar_tag(tag: str) -> str:
    """Remove namespace XML das tags."""
    return tag.split('}')[-1] if '}' in tag else tag


def converter_valor(texto: Union[str, float, int, None]) -> float:
    """
    Converte string no formato brasileiro para float com validação robusta.

    Args:
        texto: Valor a ser convertido

    Returns:
        float: Valor convertido ou 0.0 se inválido
    """
    if texto is None or texto == '':
        return 0.0

    # Se já é número, retorna
    if isinstance(texto, (int, float)):
        return float(texto)

    try:
        texto_str = str(texto).strip()
        if not texto_str:
            return 0.0

        # Remove pontos (separador de milhar) e troca vírgula por ponto
        texto_limpo = texto_str.replace('.', '').replace(',', '.')
        return float(texto_limpo)
    except (ValueError, AttributeError) as e:
        logger.debug(f"Erro ao converter '{texto}': {e}")
        return 0.0


def buscar_valor_xml(root: ET.Element, caminho: str) -> Union[float, str]:
    """
    Busca um valor no XML com tratamento robusto de namespaces.

    Args:
        root: Elemento raiz do XML
        caminho: Caminho XPath simplificado

    Returns:
        float ou string, com fallback para 0.0 ou ''
    """
    try:
        elemento = root.find(f'.//{caminho}')

        if elemento is not None and elemento.text:
            texto = elemento.text.strip()

            if not texto:
                return 0.0 if '/' in caminho else ''

            # Tentar converter para número
            try:
                return converter_valor(texto)
            except:
                # Se falhar, retornar como string
                return texto

        # Retornar valor padrão baseado no tipo esperado
        return 0.0 if '/' in caminho else ''

    except Exception as e:
        logger.debug(f"Erro ao buscar '{caminho}': {e}")
        return 0.0 if '/' in caminho else ''


# ═════════════════════════════════════════════════════════════════════
# CÉLULA 4: FUNÇÕES DE CACHE
# ═════════════════════════════════════════════════════════════════════

def gerar_cache_key(cnpj: str, doc_id: str) -> str:
    """Gera chave única para cache."""
    return hashlib.md5(f"{cnpj}_{doc_id}".encode()).hexdigest()


def salvar_cache(cnpj: str, doc_id: str, dados: Dict) -> None:
    """Salva resultado em cache."""
    if not CACHE_ENABLED:
        return

    try:
        cache_key = gerar_cache_key(cnpj, doc_id)
        cache_file = CACHE_DIR / f"{cache_key}.pkl"

        with open(cache_file, 'wb') as f:
            pickle.dump(dados, f)

        logger.debug(f"Cache salvo: {cnpj} | doc {doc_id}")
    except Exception as e:
        logger.warning(f"Erro ao salvar cache: {e}")


def ler_cache(cnpj: str, doc_id: str) -> Optional[Dict]:
    """Lê resultado do cache se existir."""
    if not CACHE_ENABLED:
        return None

    try:
        cache_key = gerar_cache_key(cnpj, doc_id)
        cache_file = CACHE_DIR / f"{cache_key}.pkl"

        if cache_file.exists():
            with open(cache_file, 'rb') as f:
                dados = pickle.load(f)
            logger.debug(f"Cache hit: {cnpj} | doc {doc_id}")
            return dados

        return None
    except Exception as e:
        logger.warning(f"Erro ao ler cache: {e}")
        return None


# ═════════════════════════════════════════════════════════════════════
# CÉLULA 5: REQUISIÇÕES COM RETRY LOGIC
# ═════════════════════════════════════════════════════════════════════

@retry(
    stop=stop_after_attempt(MAX_RETRIES),
    wait=wait_exponential(multiplier=1, min=2, max=10),
    retry=retry_if_exception_type((requests.exceptions.RequestException, requests.exceptions.Timeout)),
    reraise=True
)
def fazer_requisicao_com_retry(url: str, **kwargs) -> requests.Response:
    """
    Faz requisição HTTP com retry automático e backoff exponencial.

    Args:
        url: URL para requisição
        **kwargs: Argumentos para requests.get()

    Returns:
        Response object

    Raises:
        requests.exceptions.RequestException: Após MAX_RETRIES tentativas
    """
    logger.debug(f"Requisição: {url[:80]}...")
    response = requests.get(url, **kwargs)
    response.raise_for_status()
    return response


# ═════════════════════════════════════════════════════════════════════
# CÉLULA 6: EXTRAÇÃO DE DADOS DO XML (COM BUG FIX)
# ═════════════════════════════════════════════════════════════════════

def extrair_dados_xml_completo(xml_content: bytes) -> Dict[str, Union[str, float, int, None]]:
    """
    Extrai TODOS os campos relevantes do XML de Informe Mensal FIDC.

    CORRIGIDO: Bug de conversão de tipos no cálculo de inadimplência.

    Args:
        xml_content: Conteúdo XML em bytes (já decodificado do Base64)

    Returns:
        Dicionário com 90+ campos estruturados
    """
    try:
        root = ET.fromstring(xml_content)
    except ET.ParseError as e:
        logger.error(f"Erro ao fazer parse do XML: {e}")
        raise

    dados = {
        # ═══ SEÇÃO 1: IDENTIFICAÇÃO DO FUNDO ═══
        'CNPJ_FUNDO': buscar_valor_xml(root, 'NR_CNPJ_FUNDO'),
        'CNPJ_ADMINISTRADOR': buscar_valor_xml(root, 'NR_CNPJ_ADM'),
        'DATA_COMPETENCIA': buscar_valor_xml(root, 'DT_COMPT'),
        'TIPO_CONDOMINIO': buscar_valor_xml(root, 'TP_CONDOMINIO'),
        'FUNDO_EXCLUSIVO': buscar_valor_xml(root, 'FDO_EXCL'),
        'CLASSE_UNICA': buscar_valor_xml(root, 'CLASS_UNICA'),
        'COTISTA_VINCULADO': buscar_valor_xml(root, 'COTST_VINCUL'),

        # ═══ SEÇÃO 2: ATIVOS GERAIS ═══
        'ATIVO_TOTAL': buscar_valor_xml(root, 'VL_SOM_APLIC_ATIVO'),
        'DISPONIBILIDADES': buscar_valor_xml(root, 'VL_DISPONIB'),
        'CARTEIRA_TOTAL': buscar_valor_xml(root, 'VL_CARTEIRA'),
        'OUTROS_ATIVOS_TOTAL': buscar_valor_xml(root, 'VL_SOM_OUTROS_ATIVOS'),
        'OUTROS_ATIVOS_CURTO_PRAZO': buscar_valor_xml(root, 'OUTROS_ATIVOS/VL_OUTRO_VL_RECEB_CURPRZ'),
        'OUTROS_ATIVOS_LONGO_PRAZO': buscar_valor_xml(root, 'OUTROS_ATIVOS/VL_OUTRO_VL_RECEB_LPRAZO'),

        # ═══ SEÇÃO 3: CRÉDITOS EXISTENTES ═══
        'CREDITOS_ADQUIRIDOS': buscar_valor_xml(root, 'CRED_EXISTE/VL_SOM_DICRED_AQUIS'),
        'CRED_VENCIDOS_ADIMPLENTES': buscar_valor_xml(root, 'CRED_EXISTE/VL_CRED_EXISTE_VENC_ADIMPL'),
        'CRED_VENCIDOS_INADIMPLENTES': buscar_valor_xml(root, 'CRED_EXISTE/VL_CRED_EXISTE_VENC_INAD'),
        'CRED_TOTAL_VENC_INADIMPL': buscar_valor_xml(root, 'CRED_EXISTE/VL_CRED_TOTAL_VENC_INAD'),
        'CRED_INADIMPLENCIA': buscar_valor_xml(root, 'CRED_EXISTE/VL_CRED_EXISTE_INAD'),
        'CRED_PERFORMADOS': buscar_valor_xml(root, 'CRED_EXISTE/VL_CRED_REFER_DICRED_PERFO'),
        'CRED_VENCIDOS_PENDENTES': buscar_valor_xml(root, 'CRED_EXISTE/VL_CRED_VENC_PEND'),
        'CRED_EMP_RECUPERACAO': buscar_valor_xml(root, 'CRED_EXISTE/VL_CRED_ORIGEM_EMP_PROC_RECUP'),
        'CRED_RECEITA_PUBLICA': buscar_valor_xml(root, 'CRED_EXISTE/VL_DECOR_RECEIT_PUBLIC'),
        'CRED_ACAO_JUDICIAL': buscar_valor_xml(root, 'CRED_EXISTE/VL_CRED_ACAO_JUDIC'),
        'CRED_CONSTITUICAO_JURIDICA': buscar_valor_xml(root, 'CRED_EXISTE/VL_CRED_CONST_JUR_FATRISC'),
        'CRED_PROVISAO_REDUCAO': buscar_valor_xml(root, 'CRED_EXISTE/VL_PROVIS_REDUC_RECUP'),

        # ═══ SEÇÃO 4: DIREITOS CREDITÓRIOS (DICRED) ═══
        'DICRED_TOTAL': buscar_valor_xml(root, 'DICRED/VL_DICRED'),
        'DICRED_CEDENTE': buscar_valor_xml(root, 'DICRED/VL_DICRED_CEDENT'),
        'DICRED_VENC_INADIMPL': buscar_valor_xml(root, 'DICRED/VL_DICRED_EXISTE_VENC_INAD'),
        'DICRED_TOTAL_VENC_INAD': buscar_valor_xml(root, 'DICRED/VL_DICRED_TOTAL_VENC_INAD'),
        'DICRED_INADIMPLENCIA': buscar_valor_xml(root, 'DICRED/VL_DICRED_EXISTE_INAD'),
        'DICRED_PERFORMADOS': buscar_valor_xml(root, 'DICRED/VL_DICRED_REFER_DICRED_PERFO'),
        'DICRED_VENC_PENDENTES': buscar_valor_xml(root, 'DICRED/VL_DICRED_VENC_PEND'),
        'DICRED_EMP_RECUPERACAO': buscar_valor_xml(root, 'DICRED/VL_DICRED_ORIGEM_EMP_PROC_RECUP'),
        'DICRED_RECEITA_PUBLICA': buscar_valor_xml(root, 'DICRED/VL_DICRED_RECEIT_PUBLIC'),
        'DICRED_ACAO_JUDICIAL': buscar_valor_xml(root, 'DICRED/VL_DICRED_ACAO_JUDIC'),
        'DICRED_PROVISAO_REDUCAO': buscar_valor_xml(root, 'DICRED/VL_DICRED_PROVIS_REDUC_RECUP'),

        # ═══ SEÇÃO 5: VALORES MOBILIÁRIOS ═══
        'VALORES_MOBILIARIOS_TOTAL': buscar_valor_xml(root, 'VALORES_MOB/VL_SOM_VALORES_MOB'),
        'DEBENTURES': buscar_valor_xml(root, 'VALORES_MOB/VL_DEBT'),
        'CRI': buscar_valor_xml(root, 'VALORES_MOB/VL_CRI'),
        'NOTAS_PROMISSORIAS_COMERCIAIS': buscar_valor_xml(root, 'VALORES_MOB/VL_NP_COMERC'),
        'LETRAS_FINANCEIRAS': buscar_valor_xml(root, 'VALORES_MOB/VL_LETRA_FINANC'),
        'COTAS_FIF': buscar_valor_xml(root, 'VALORES_MOB/VL_CLS_COTA_FIF'),
        'OUTROS_DIREITOS_CREDITORIOS': buscar_valor_xml(root, 'VALORES_MOB/VL_OUTRO_DICRED'),

        # ═══ SEÇÃO 6: OUTROS ATIVOS FINANCEIROS ═══
        'TITULOS_PUBLICOS_FEDERAIS': buscar_valor_xml(root, 'VL_TITPUB_FED'),
        'CDB': buscar_valor_xml(root, 'VL_CDB'),
        'APLICACOES_COMPROMISSADAS': buscar_valor_xml(root, 'VL_APLIC_OPER_COMPSS'),
        'ATIVOS_FINANCEIROS_RF': buscar_valor_xml(root, 'VL_ATIV_FINANC_RF'),
        'COTAS_FIDC': buscar_valor_xml(root, 'VL_COTA_FIDC'),

        # ═══ SEÇÃO 7: MERCADO DE DERIVATIVOS ═══
        'DERIVATIVOS_TOTAL': buscar_valor_xml(root, 'MERC_DERIVATIVO/VL_SOM_MERC_DERIVATIVO'),
        'TERMO_COMPRADOR': buscar_valor_xml(root, 'MERC_DERIVATIVO/VL_MERC_TERMO_POS_COMPRD'),
        'OPCOES_TITULAR': buscar_valor_xml(root, 'MERC_DERIVATIVO/VL_MERC_OP_POS_TITUL'),
        'FUTUROS_AJUSTE_POSITIVO': buscar_valor_xml(root, 'MERC_DERIVATIVO/VL_MERC_FUT_AJUST_POSIT'),
        'SWAP_A_RECEBER': buscar_valor_xml(root, 'MERC_DERIVATIVO/VL_DIFER_SWAP_RECEB'),
        'COBERTURA_PRESTADA': buscar_valor_xml(root, 'MERC_DERIVATIVO/VL_COBERT_PREST'),
        'DEPOSITOS_MARGEM': buscar_valor_xml(root, 'MERC_DERIVATIVO/VL_DEPOS_MARGEM'),

        # ═══ SEÇÃO 8: SEGMENTAÇÃO DA CARTEIRA ═══
        'CARTEIRA_SEGMENTADA_TOTAL': buscar_valor_xml(root, 'CART_SEGMT/VL_SOM_CART_SEGMT'),
        'SEGMT_INDUSTRIAL': buscar_valor_xml(root, 'CART_SEGMT/VL_IND'),
        'SEGMT_MERCADO_IMOBILIARIO': buscar_valor_xml(root, 'CART_SEGMT/VL_MERC_IMOBIL'),
        'SEGMT_AGRONEGOCIO': buscar_valor_xml(root, 'CART_SEGMT/VL_AGRONEG'),
        'SEGMT_CARTAO_CREDITO': buscar_valor_xml(root, 'CART_SEGMT/VL_CART_CRED'),
        'SEGMT_ACAO_JUDICIAL': buscar_valor_xml(root, 'CART_SEGMT/VL_ACAO_JUDIC'),
        'SEGMT_PROPRIEDADE_INTELECTUAL': buscar_valor_xml(root, 'CART_SEGMT/VL_PROPRD_MARCA_PATENT'),

        # Subsegmento: COMERCIAL
        'SEGMT_COMERCIAL_TOTAL': buscar_valor_xml(root, 'SEGMT_COMERC/VL_SOM_SEGMT_COMERC'),
        'SEGMT_COMERCIO': buscar_valor_xml(root, 'SEGMT_COMERC/VL_COMERC'),
        'SEGMT_COMERCIO_VAREJO': buscar_valor_xml(root, 'SEGMT_COMERC/VL_COMERC_VARJ'),
        'SEGMT_ARREND_MERCANTIL': buscar_valor_xml(root, 'SEGMT_COMERC/VL_ARREND_MERCNT'),

        # Subsegmento: SERVIÇOS
        'SEGMT_SERVICOS_TOTAL': buscar_valor_xml(root, 'SEGMT_SERV/VL_SOM_SEGMT_SERV'),
        'SEGMT_SERVICOS_GERAIS': buscar_valor_xml(root, 'SEGMT_SERV/VL_SERV'),
        'SEGMT_SERVICOS_PUBLICOS': buscar_valor_xml(root, 'SEGMT_SERV/VL_SERV_PUBLIC'),
        'SEGMT_SERVICOS_EDUCACAO': buscar_valor_xml(root, 'SEGMT_SERV/VL_SERV_EDUC'),
        'SEGMT_SERVICOS_ENTRETENIMENTO': buscar_valor_xml(root, 'SEGMT_SERV/VL_SERV_ENTRETEN'),

        # Subsegmento: FINANCEIRO
        'SEGMT_FINANCEIRO_TOTAL': buscar_valor_xml(root, 'SEGMT_FINANC/VL_SOM_SEGMT_FINANC'),
        'SEGMT_FINANC_CREDITO_PESSOA': buscar_valor_xml(root, 'SEGMT_FINANC/VL_FINANC_CRED_PESSOA'),
        'SEGMT_FINANC_CONSIGNADO': buscar_valor_xml(root, 'SEGMT_FINANC/VL_FINANC_CRED_PESSOA_CONSIG'),
        'SEGMT_FINANC_CORPORATIVO': buscar_valor_xml(root, 'SEGMT_FINANC/VL_FINANC_CRED_CORPOR'),
        'SEGMT_FINANC_MIDDLE_MARKET': buscar_valor_xml(root, 'SEGMT_FINANC/VL_FINANC_MMARKET'),
        'SEGMT_FINANC_VEICULOS': buscar_valor_xml(root, 'SEGMT_FINANC/VL_FINANC_VEICL'),
        'SEGMT_FINANC_IMOB_EMPRESARIAL': buscar_valor_xml(root, 'SEGMT_FINANC/VL_FINANC_IMOBIL_EMPSRL'),
        'SEGMT_FINANC_IMOB_RESIDENCIAL': buscar_valor_xml(root, 'SEGMT_FINANC/VL_FINANC_IMOBIL_RESID'),
        'SEGMT_FINANC_OUTROS': buscar_valor_xml(root, 'SEGMT_FINANC/VL_FINANC_OUTRO'),

        # Subsegmento: FACTORING
        'SEGMT_FACTORING_TOTAL': buscar_valor_xml(root, 'SEGMT_FACT/VL_SOM_SEGMT_FACT'),
        'SEGMT_FACTORING_PESSOA': buscar_valor_xml(root, 'SEGMT_FACT/VL_FACT_PESSOA'),
        'SEGMT_FACTORING_CORPORATIVO': buscar_valor_xml(root, 'SEGMT_FACT/VL_FACT_CORPOR'),

        # Subsegmento: SETOR PÚBLICO
        'SEGMT_SETOR_PUBLICO_TOTAL': buscar_valor_xml(root, 'SEGMT_SETOR_PUBLIC/VL_SOM_SEGMT_SETOR_PUBLIC'),
        'SEGMT_PRECATORIOS': buscar_valor_xml(root, 'SEGMT_SETOR_PUBLIC/VL_SETOR_PUBLIC_PRECAT'),
        'SEGMT_CREDITOS_TRIBUTARIOS': buscar_valor_xml(root, 'SEGMT_SETOR_PUBLIC/VL_SETOR_PUBLIC_CRED_TRIBUT'),
        'SEGMT_ROYALTIES': buscar_valor_xml(root, 'SEGMT_SETOR_PUBLIC/VL_SETOR_PUBLIC_ROYA'),
        'SEGMT_SETOR_PUBLICO_OUTROS': buscar_valor_xml(root, 'SEGMT_SETOR_PUBLIC/VL_SETOR_PUBLIC_OUTRO'),
    }

    # ─────────────────────────────────────────────────────────────────────────
    # ✅ BUG FIX: CÁLCULO DE INDICADORES COM CONVERSÃO FORÇADA
    # ─────────────────────────────────────────────────────────────────────────

    # Garantir que valores são float antes de comparar
    inadimpl_cred = float(dados.get('CRED_INADIMPLENCIA', 0) or 0)
    inadimpl_dicred = float(dados.get('DICRED_INADIMPLENCIA', 0) or 0)
    dados['INADIMPLENCIA_TOTAL'] = max(inadimpl_cred, inadimpl_dicred)

    carteira_credito = float(dados.get('CREDITOS_ADQUIRIDOS', 0) or 0)
    ativo_total = float(dados.get('ATIVO_TOTAL', 0) or 0)
    disponibilidades = float(dados.get('DISPONIBILIDADES', 0) or 0)

    # Índice de NPL
    if carteira_credito > 0 and dados['INADIMPLENCIA_TOTAL'] > 0:
        dados['INDICE_NPL_PERCENTUAL'] = (dados['INADIMPLENCIA_TOTAL'] / carteira_credito) * 100
    else:
        dados['INDICE_NPL_PERCENTUAL'] = 0.0

    # Taxa de liquidez
    if ativo_total > 0:
        dados['TAXA_LIQUIDEZ_PERCENTUAL'] = (disponibilidades / ativo_total) * 100
    else:
        dados['TAXA_LIQUIDEZ_PERCENTUAL'] = 0.0

    # Concentração em crédito
    if ativo_total > 0:
        dados['CONCENTRACAO_CREDITO_PERCENTUAL'] = (carteira_credito / ativo_total) * 100
    else:
        dados['CONCENTRACAO_CREDITO_PERCENTUAL'] = 0.0

    # Validação de NPL outlier
    if dados['INDICE_NPL_PERCENTUAL'] > 1000:
        logger.warning(f"NPL outlier detectado: {dados['INDICE_NPL_PERCENTUAL']:.2f}% | CNPJ: {dados.get('CNPJ_FUNDO', 'N/A')}")
        dados['NPL_OUTLIER_FLAG'] = True
    else:
        dados['NPL_OUTLIER_FLAG'] = False

    return dados


logger.info("✅ Função de extração definida (90+ campos + bug fix)")

# ═════════════════════════════════════════════════════════════════════
# CÉLULA 7: FUNÇÃO ETL PRINCIPAL COM PARALELIZAÇÃO
# ═════════════════════════════════════════════════════════════════════

def etl_fidc_historico(
    cnpj_alvo: str,
    nome_fundo_referencia: str = "",
    anos_dezembro: List[int] = None,
    coletar_ultimo: bool = True
) -> List[Dict]:
    """
    Executa ETL completo para um CNPJ com cache e retry.

    Args:
        cnpj_alvo: CNPJ do fundo (14 dígitos)
        nome_fundo_referencia: Nome do fundo (opcional)
        anos_dezembro: Anos para buscar dezembro
        coletar_ultimo: Se True, coleta último mês

    Returns:
        Lista de dicionários com dados extraídos
    """

    resultados_fundo_cnpj = []

    if anos_dezembro is None:
        anos_dezembro = ANOS_DEZEMBRO

    try:
        # ─────────────────────────────────────────────────────────────────────
        # ETAPA 1: DISCOVERY (COM RETRY)
        # ─────────────────────────────────────────────────────────────────────

        params = {
            'd': 0,
            's': 0,
            'l': 200,
            'cnpjFundo': cnpj_alvo
        }

        resp_busca = fazer_requisicao_com_retry(
            URL_API_BUSCA,
            params=params,
            headers=HEADERS,
            timeout=TIMEOUT_BUSCA
        )

        data = resp_busca.json().get('data', [])

        if not data:
            logger.debug(f"CNPJ {cnpj_alvo}: Nenhum documento encontrado")
            return []

        df_docs = pd.DataFrame(data)

        # ─────────────────────────────────────────────────────────────────────
        # ETAPA 2: FILTER
        # ─────────────────────────────────────────────────────────────────────

        df_mensal = df_docs[
            (df_docs['tipoDocumento'].str.strip() == "Informe Mensal Estruturado") &
            (df_docs['situacaoDocumento'].str.strip() == "A")
        ].copy()

        if df_mensal.empty:
            logger.debug(f"CNPJ {cnpj_alvo}: Nenhum informe mensal ativo")
            return []

        # Converter datas
        df_mensal['dataReferenciaOrdenavel'] = pd.to_datetime(
            df_mensal['dataReferencia'].str[3:] + '-' +
            df_mensal['dataReferencia'].str[:2] + '-01',
            format='%Y-%m-%d', errors='coerce'
        )

        df_mensal.dropna(subset=['dataReferenciaOrdenavel'], inplace=True)

        if df_mensal.empty:
            logger.debug(f"CNPJ {cnpj_alvo}: Datas inválidas")
            return []

        # ─────────────────────────────────────────────────────────────────────
        # ETAPA 3: SELEÇÃO DE DOCUMENTOS
        # ─────────────────────────────────────────────────────────────────────

        documentos_processar = []

        # Último mês
        if coletar_ultimo:
            df_mensal_sorted = df_mensal.sort_values('dataReferenciaOrdenavel', ascending=False)
            df_ultimo = df_mensal_sorted.groupby('descricaoFundo').first().reset_index()
            documentos_processar.extend(df_ultimo.to_dict('records'))

        # Dezembros
        for ano in anos_dezembro:
            data_dezembro = f"12/{ano}"
            docs_dezembro = df_mensal[df_mensal['dataReferencia'] == data_dezembro]

            if not docs_dezembro.empty:
                df_dez_unico = docs_dezembro.groupby('descricaoFundo').first().reset_index()
                documentos_processar.extend(df_dez_unico.to_dict('records'))

        # Remover duplicatas por ID
        ids_processados = set()
        docs_unicos = []
        for doc in documentos_processar:
            doc_id = doc['id']
            if doc_id not in ids_processados:
                docs_unicos.append(doc)
                ids_processados.add(doc_id)

        logger.debug(f"CNPJ {cnpj_alvo}: {len(docs_unicos)} documentos para processar")

        # ─────────────────────────────────────────────────────────────────────
        # ETAPA 4-6: DOWNLOAD, EXTRACT, CLEANUP (COM CACHE)
        # ─────────────────────────────────────────────────────────────────────

        for doc_info in docs_unicos:
            doc_id = str(doc_info['id'])
            data_referencia = doc_info.get('dataReferencia', 'N/A')
            nome_fundo_doc = doc_info.get('descricaoFundo', 'Nome N/A')

            # Tentar ler do cache primeiro
            dados_cache = ler_cache(cnpj_alvo, doc_id)
            if dados_cache is not None:
                resultados_fundo_cnpj.append(dados_cache)
                continue

            nome_arquivo_temp = None

            try:
                # Download com retry
                url_download = f"{URL_API_DOWNLOAD}?id={doc_id}"
                resp_download = fazer_requisicao_com_retry(
                    url_download,
                    headers=HEADERS,
                    timeout=TIMEOUT_DOWNLOAD
                )

                xml_content = base64.b64decode(resp_download.content)

                # Salvar temporariamente
                nome_arquivo_temp = f'temp_{cnpj_alvo}_{doc_id}.xml'
                with open(nome_arquivo_temp, 'wb') as f:
                    f.write(xml_content)

                # Extrair dados
                dados_extraidos = extrair_dados_xml_completo(xml_content)

                if not dados_extraidos.get('CNPJ_FUNDO'):
                    dados_extraidos['CNPJ_FUNDO'] = cnpj_alvo

                # Metadados
                dados_extraidos['NOME_FUNDO'] = nome_fundo_doc
                dados_extraidos['STATUS'] = 'SUCESSO'
                dados_extraidos['ID_DOCUMENTO'] = doc_id
                dados_extraidos['DATA_REFERENCIA_DOC'] = data_referencia
                dados_extraidos['MENSAGEM_ERRO'] = None

                # Tipo de coleta
                if data_referencia.startswith('12/'):
                    ano = data_referencia.split('/')[1]
                    dados_extraidos['TIPO_COLETA'] = f'DEZEMBRO_{ano}'
                else:
                    dados_extraidos['TIPO_COLETA'] = 'ULTIMO_MES'

                # Salvar em cache
                salvar_cache(cnpj_alvo, doc_id, dados_extraidos)

                resultados_fundo_cnpj.append(dados_extraidos)

            except requests.exceptions.HTTPError as e:
                logger.warning(f"HTTP Error | CNPJ {cnpj_alvo} | Doc {doc_id}: {e}")
                resultados_fundo_cnpj.append({
                    'CNPJ_FUNDO': cnpj_alvo,
                    'NOME_FUNDO': nome_fundo_doc,
                    'STATUS': 'ERRO_HTTP_DOWNLOAD',
                    'MENSAGEM_ERRO': f'HTTP Error: {str(e)}',
                    'DATA_REFERENCIA_DOC': data_referencia,
                    'TIPO_COLETA': 'ERRO'
                })
            except requests.exceptions.Timeout as e:
                logger.warning(f"Timeout | CNPJ {cnpj_alvo} | Doc {doc_id}: {e}")
                resultados_fundo_cnpj.append({
                    'CNPJ_FUNDO': cnpj_alvo,
                    'NOME_FUNDO': nome_fundo_doc,
                    'STATUS': 'TIMEOUT_DOWNLOAD',
                    'MENSAGEM_ERRO': f'Timeout: {str(e)}',
                    'DATA_REFERENCIA_DOC': data_referencia,
                    'TIPO_COLETA': 'ERRO'
                })
            except ET.ParseError as e:
                logger.error(f"Parse Error | CNPJ {cnpj_alvo} | Doc {doc_id}: {e}")
                resultados_fundo_cnpj.append({
                    'CNPJ_FUNDO': cnpj_alvo,
                    'NOME_FUNDO': nome_fundo_doc,
                    'STATUS': 'ERRO_PARSE_XML',
                    'MENSAGEM_ERRO': f'Erro XML: {str(e)}',
                    'DATA_REFERENCIA_DOC': data_referencia,
                    'TIPO_COLETA': 'ERRO'
                })
            except Exception as e:
                logger.error(f"Erro inesperado | CNPJ {cnpj_alvo} | Doc {doc_id}: {e}", exc_info=True)
                resultados_fundo_cnpj.append({
                    'CNPJ_FUNDO': cnpj_alvo,
                    'NOME_FUNDO': nome_fundo_doc,
                    'STATUS': 'ERRO_INESPERADO',
                    'MENSAGEM_ERRO': f'Erro: {str(e)}',
                    'DATA_REFERENCIA_DOC': data_referencia,
                    'TIPO_COLETA': 'ERRO'
                })

            finally:
                # Cleanup
                if nome_arquivo_temp and os.path.exists(nome_arquivo_temp):
                    try:
                        os.remove(nome_arquivo_temp)
                    except OSError:
                        pass

    except Exception as e:
        logger.error(f"Erro geral | CNPJ {cnpj_alvo}: {e}", exc_info=True)
        resultados_fundo_cnpj.append({
            'CNPJ_FUNDO': cnpj_alvo,
            'NOME_FUNDO': nome_fundo_referencia,
            'STATUS': 'ERRO_GERAL',
            'MENSAGEM_ERRO': f'Erro geral: {str(e)}',
            'TIPO_COLETA': 'ERRO'
        })

    return resultados_fundo_cnpj


logger.info("✅ Função ETL com retry, cache e paralelização pronta")

# ═════════════════════════════════════════════════════════════════════
# CÉLULA 8: CARREGAMENTO DOS CNPJs
# ═════════════════════════════════════════════════════════════════════

ARQUIVO_CSV_ENTRADA = 'lista_cnpjs_fidc.csv'

try:
    df_cnpjs_entrada = pd.read_csv(ARQUIVO_CSV_ENTRADA, encoding='utf-8-sig')

    logger.info(f"✅ Arquivo '{ARQUIVO_CSV_ENTRADA}' carregado")
    logger.info(f"   Total de CNPJs: {len(df_cnpjs_entrada)}")

    df_cnpjs_entrada['CNPJ'] = df_cnpjs_entrada['CNPJ'].astype(str).str.replace('.0', '', regex=False)
    df_cnpjs_entrada['CNPJ'] = df_cnpjs_entrada['CNPJ'].str.zfill(14)

    # Estimativa de tempo
    docs_estimados_por_cnpj = (1 if COLETAR_ULTIMO_MES else 0) + len(ANOS_DEZEMBRO)
    tempo_estimado_seq = len(df_cnpjs_entrada) * DELAY_ENTRE_REQUISICOES * docs_estimados_por_cnpj / 60
    tempo_estimado_par = tempo_estimado_seq / MAX_WORKERS

    logger.info(f"⏱️  Tempo estimado:")
    logger.info(f"   Sequencial: ~{tempo_estimado_seq:.1f} min")
    logger.info(f"   Paralelo ({MAX_WORKERS} threads): ~{tempo_estimado_par:.1f} min")
    logger.info(f"   Ganho: {(1 - tempo_estimado_par/tempo_estimado_seq)*100:.0f}%")

except FileNotFoundError:
    logger.error(f"❌ Arquivo '{ARQUIVO_CSV_ENTRADA}' não encontrado!")
    raise

# ═════════════════════════════════════════════════════════════════════
# CÉLULA 9: PROCESSAMENTO EM LOTE COM PARALELIZAÇÃO
# ═════════════════════════════════════════════════════════════════════

logger.info("="*80)
logger.info("🚀 INICIANDO PROCESSAMENTO PARALELO")
logger.info("="*80)

resultados_consolidados = []
tempo_inicio = datetime.now()

# Processar com ThreadPoolExecutor
with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
    # Submeter todas as tarefas
    future_to_cnpj = {}
    for idx, row in df_cnpjs_entrada.iterrows():
        cnpj = row['CNPJ']
        nome_fundo_ref = row.get('NOME_FUNDO', '')

        future = executor.submit(
            etl_fidc_historico,
            cnpj,
            nome_fundo_ref,
            ANOS_DEZEMBRO,
            COLETAR_ULTIMO_MES
        )
        future_to_cnpj[future] = (cnpj, nome_fundo_ref)

    # Processar conforme completam
    with tqdm(total=len(df_cnpjs_entrada), desc="Processando CNPJs", unit="cnpj") as pbar:
        for future in as_completed(future_to_cnpj):
            cnpj, nome_fundo_ref = future_to_cnpj[future]

            try:
                resultados_atuais = future.result()

                if resultados_atuais:
                    resultados_consolidados.extend(resultados_atuais)

                    # Log sucesso
                    sucessos = [r for r in resultados_atuais if r.get('STATUS') == 'SUCESSO']
                    if sucessos:
                        num_docs = len(sucessos)
                        pbar.set_postfix({'últimos_docs': num_docs, 'status': '✅'})
                    else:
                        pbar.set_postfix({'status': '❌ Erro'})
                else:
                    resultados_consolidados.append({
                        'CNPJ_FUNDO': cnpj,
                        'NOME_FUNDO': nome_fundo_ref,
                        'STATUS': 'NENHUM_DOCUMENTO',
                        'TIPO_COLETA': 'ERRO'
                    })
                    pbar.set_postfix({'status': '❌ Sem docs'})

            except Exception as e:
                logger.error(f"Erro ao processar futuro de {cnpj}: {e}")
                pbar.set_postfix({'status': '❌ Exception'})

            pbar.update(1)

            # Rate limiting suave entre completions
            time.sleep(0.1)

tempo_decorrido = (datetime.now() - tempo_inicio).total_seconds()

logger.info("="*80)
logger.info("✅ PROCESSAMENTO CONCLUÍDO!")
logger.info(f"   Tempo decorrido: {tempo_decorrido/60:.2f} minutos")
logger.info("="*80)

# ═════════════════════════════════════════════════════════════════════
# CÉLULA 10: CONSOLIDAÇÃO E ESTATÍSTICAS
# ═════════════════════════════════════════════════════════════════════

df_resultado_final = pd.DataFrame(resultados_consolidados)

total_documentos = len(df_resultado_final)
total_sucesso = len(df_resultado_final[df_resultado_final['STATUS'] == 'SUCESSO'])
total_erros = total_documentos - total_sucesso

logger.info("📊 ESTATÍSTICAS DO PROCESSAMENTO")
logger.info("-"*80)
logger.info(f"   Total de CNPJs processados: {len(df_cnpjs_entrada)}")
logger.info(f"   Total de documentos coletados: {total_documentos}")
logger.info(f"   ✅ Sucesso: {total_sucesso} ({total_sucesso/total_documentos*100:.1f}%)")
logger.info(f"   ❌ Erros: {total_erros} ({total_erros/total_documentos*100:.1f}%)")

# Distribuição por tipo
if total_sucesso > 0:
    df_sucesso = df_resultado_final[df_resultado_final['STATUS'] == 'SUCESSO'].copy()

    logger.info("\n📅 DISTRIBUIÇÃO POR TIPO DE COLETA:")
    tipos_coleta = df_sucesso['TIPO_COLETA'].value_counts()
    for tipo, count in tipos_coleta.items():
        logger.info(f"   • {tipo}: {count} documentos")

# Distribuição de erros
if total_erros > 0:
    logger.info("\n⚠️  TIPOS DE ERROS:")
    erros_por_tipo = df_resultado_final[df_resultado_final['STATUS'] != 'SUCESSO']['STATUS'].value_counts()
    for status, count in erros_por_tipo.items():
        logger.info(f"   • {status}: {count}")

# Estatísticas financeiras
if total_sucesso > 0:
    for col in ['ATIVO_TOTAL', 'INADIMPLENCIA_TOTAL', 'INDICE_NPL_PERCENTUAL']:
        df_sucesso[col] = pd.to_numeric(df_sucesso[col], errors='coerce')

    df_sucesso_valido = df_sucesso[df_sucesso['ATIVO_TOTAL'] > 0]

    if len(df_sucesso_valido) > 0:
        logger.info("\n💰 ESTATÍSTICAS FINANCEIRAS")
        logger.info("-"*80)

        ativo_total_soma = df_sucesso_valido['ATIVO_TOTAL'].sum()
        ativo_total_media = df_sucesso_valido['ATIVO_TOTAL'].mean()
        inadimpl_soma = df_sucesso_valido['INADIMPLENCIA_TOTAL'].sum()
        npl_medio = df_sucesso_valido['INDICE_NPL_PERCENTUAL'].mean()

        logger.info(f"   Ativo Total (Soma):     R$ {ativo_total_soma:,.2f}")
        logger.info(f"   Ativo Total (Média):    R$ {ativo_total_media:,.2f}")
        logger.info(f"   Inadimplência (Soma):   R$ {inadimpl_soma:,.2f}")
        logger.info(f"   NPL Médio:              {npl_medio:.2f}%")

# ═════════════════════════════════════════════════════════════════════
# CÉLULA 11: EXPORTAÇÃO
# ═════════════════════════════════════════════════════════════════════

# CSV principal
arquivo_csv = 'base_fidc_completa_historica_v5.csv'
df_resultado_final.to_csv(arquivo_csv, index=False, encoding='utf-8-sig', sep=';', decimal=',')
logger.info(f"\n💾 Arquivo salvo: {arquivo_csv}")

# Excel com múltiplas abas
arquivo_excel = 'base_fidc_completa_historica_v5.xlsx'

try:
    with pd.ExcelWriter(arquivo_excel, engine='openpyxl') as writer:
        df_resultado_final.to_excel(writer, sheet_name='Dados_Completos', index=False)

        if total_sucesso > 0:
            df_sucesso.to_excel(writer, sheet_name='Dados_Validos', index=False)

        if total_erros > 0:
            df_erros = df_resultado_final[df_resultado_final['STATUS'] != 'SUCESSO']
            df_erros.to_excel(writer, sheet_name='Erros', index=False)

        if COLETAR_ULTIMO_MES and total_sucesso > 0:
            df_ultimo = df_sucesso[df_sucesso['TIPO_COLETA'] == 'ULTIMO_MES']
            if not df_ultimo.empty:
                df_ultimo.to_excel(writer, sheet_name='Ultimo_Mes', index=False)

        for ano in ANOS_DEZEMBRO:
            tipo_coleta = f'DEZEMBRO_{ano}'
            df_ano = df_sucesso[df_sucesso['TIPO_COLETA'] == tipo_coleta]
            if not df_ano.empty:
                df_ano.to_excel(writer, sheet_name=f'Dezembro_{ano}', index=False)

    logger.info(f"💾 Arquivo salvo: {arquivo_excel}")
except Exception as e:
    logger.warning(f"Não foi possível gerar Excel: {e}")

# Log final
logger.info("\n" + "="*80)
logger.info("✅ ETL FIDC V5 CONCLUÍDO COM SUCESSO!")
logger.info("="*80)
logger.info(f"📁 Arquivos gerados:")
logger.info(f"   • {arquivo_csv}")
logger.info(f"   • {arquivo_excel}")
logger.info(f"   • {log_filename}")
logger.info(f"\n🎯 Melhorias implementadas:")
logger.info(f"   1. ✅ Bug fix - Conversão de tipos corrigida")
logger.info(f"   2. ✅ Logging profissional ({log_filename})")
logger.info(f"   3. ✅ Retry com backoff exponencial (até {MAX_RETRIES}x)")
logger.info(f"   4. ✅ Paralelização ({MAX_WORKERS} threads)")
logger.info(f"   5. ✅ Cache de resultados ({CACHE_DIR}/)")
logger.info(f"   6. ✅ Progress bar visual")
logger.info(f"   7. ✅ Validação de NPL outliers")
logger.info("\n⏱️  Performance:")
logger.info(f"   Tempo de execução: {tempo_decorrido/60:.2f} minutos")
logger.info(f"   Média por CNPJ: {tempo_decorrido/len(df_cnpjs_entrada):.2f} segundos")
logger.info(f"   Taxa de sucesso: {total_sucesso/total_documentos*100:.1f}%")
logger.info("="*80)


✅ Dependências verificadas!


Processando CNPJs:   0%|          | 0/441 [00:00<?, ?cnpj/s]

In [1]:
# ═════════════════════════════════════════════════════════════════════
# CÉLULA 8: EXPORTAR RESULTADOS PARA CSV (VERSÃO CORRIGIDA)
# ═════════════════════════════════════════════════════════════════════

print("\n" + "=" * 80)
print("📊 EXPORTANDO RESULTADOS PARA CSV")
print("=" * 80)

# ─────────────────────────────────────────────────────────────────────────
# 1. EXPORTAR RESULTADO COMPLETO (TODOS OS REGISTROS)
# ─────────────────────────────────────────────────────────────────────────

nome_arquivo_csv = 'base_fidc_completa_historica.csv'

print(f"\n📁 Exportando arquivo completo: {nome_arquivo_csv}")
print(f"   • Total de registros: {len(df_resultado_final):,}")

# ✅ CORREÇÃO: Adicionado float_format='%.2f' para evitar notação científica
df_resultado_final.to_csv(
    nome_arquivo_csv,
    index=False,
    encoding='utf-8-sig',
    sep=';',
    decimal=',',
    float_format='%.2f'  # ← NOVA LINHA: Força formato decimal sem notação científica
)

print(f"   ✅ Arquivo salvo com sucesso!")

# ─────────────────────────────────────────────────────────────────────────
# 2. EXPORTAR APENAS SUCESSOS (OPCIONAL)
# ─────────────────────────────────────────────────────────────────────────

df_sucesso = df_resultado_final[df_resultado_final['STATUS'] == 'SUCESSO'].copy()

if not df_sucesso.empty:
    nome_arquivo_sucesso = 'base_fidc_sucesso_apenas.csv'

    print(f"\n📁 Exportando apenas sucessos: {nome_arquivo_sucesso}")
    print(f"   • Registros de sucesso: {len(df_sucesso):,}")

    # ✅ CORREÇÃO: Adicionado float_format='%.2f'
    df_sucesso.to_csv(
        nome_arquivo_sucesso,
        index=False,
        encoding='utf-8-sig',
        sep=';',
        decimal=',',
        float_format='%.2f'  # ← NOVA LINHA
    )

    print(f"   ✅ Arquivo salvo com sucesso!")

# ─────────────────────────────────────────────────────────────────────────
# 3. EXPORTAR APENAS ERROS (PARA ANÁLISE)
# ─────────────────────────────────────────────────────────────────────────

df_erros = df_resultado_final[df_resultado_final['STATUS'] != 'SUCESSO'].copy()

if not df_erros.empty:
    nome_arquivo_erros = 'base_fidc_erros.csv'

    print(f"\n📁 Exportando erros: {nome_arquivo_erros}")
    print(f"   • Registros com erro: {len(df_erros):,}")

    # ✅ CORREÇÃO: Adicionado float_format='%.2f'
    df_erros.to_csv(
        nome_arquivo_erros,
        index=False,
        encoding='utf-8-sig',
        sep=';',
        decimal=',',
        float_format='%.2f'  # ← NOVA LINHA
    )

    print(f"   ✅ Arquivo salvo com sucesso!")

# ─────────────────────────────────────────────────────────────────────────
# 4. RELATÓRIO FINAL
# ─────────────────────────────────────────────────────────────────────────

print("\n" + "=" * 80)
print("✅ EXPORTAÇÃO CONCLUÍDA!")
print("=" * 80)

print(f"\n📊 RESUMO DOS ARQUIVOS GERADOS:")
print(f"   1. {nome_arquivo_csv}")
print(f"      • {len(df_resultado_final):,} registros totais")
print(f"      • {len(df_resultado_final.columns)} colunas")

if not df_sucesso.empty:
    print(f"\n   2. {nome_arquivo_sucesso}")
    print(f"      • {len(df_sucesso):,} registros de sucesso")
    print(f"      • Taxa de sucesso: {len(df_sucesso)/len(df_resultado_final)*100:.1f}%")

if not df_erros.empty:
    print(f"\n   3. {nome_arquivo_erros}")
    print(f"      • {len(df_erros):,} registros com erro")
    print(f"      • Taxa de erro: {len(df_erros)/len(df_resultado_final)*100:.1f}%")

print("\n" + "=" * 80)
print("💾 TODOS OS ARQUIVOS SALVOS SEM NOTAÇÃO CIENTÍFICA!")
print("=" * 80)

# ─────────────────────────────────────────────────────────────────────────
# 5. VERIFICAÇÃO RÁPIDA (OPCIONAL - PARA DEBUG)
# ─────────────────────────────────────────────────────────────────────────

print("\n🔍 VERIFICAÇÃO RÁPIDA DOS DADOS:")
print("-" * 80)

# Mostrar exemplo de valores grandes (verificar se não tem notação científica)
colunas_verificar = ['ATIVO_TOTAL', 'CARTEIRA_TOTAL', 'SEGMT_ACAO_JUDICIAL']

for col in colunas_verificar:
    if col in df_resultado_final.columns:
        valor_max = df_resultado_final[col].max()
        if pd.notna(valor_max) and valor_max > 0:
            print(f"   • {col}:")
            print(f"     Valor máximo: {valor_max:,.2f}".replace(',', 'X').replace('.', ',').replace('X', '.'))

print("\n✅ Se os valores acima aparecem sem 'E+', está correto!")
print("=" * 80)



📊 EXPORTANDO RESULTADOS PARA CSV

📁 Exportando arquivo completo: base_fidc_completa_historica.csv


NameError: name 'df_resultado_final' is not defined